In [1]:
import os
import pandas as pd
import numpy as np
import dalmatian as dm

In [85]:
folder = 'gs://ccle_bams/'

In [86]:
samples = os.popen('gsutil -m ls -al '+folder +'**.bam').read().split('\n')

In [91]:
sizes = {'gs://' + val.split('gs://')[1].split('#')[0]: int(val.split(" 2019-")[0] if "2019-" in val else val.split(" 2020-")[0]) for val in samples[:-2]}

In [92]:
sizes

{'gs://ccle_bams/hg38_wes/ACH-001421_2.hg38.bam': 18871404272,
 'gs://ccle_bams/hg38_wes/ACH-001533_1.hg38.bam': 29204396795,
 'gs://ccle_bams/hg38_wes/ACH-001574_1.hg38.bam': 30888654010,
 'gs://ccle_bams/hg38_wes/ACH-001678_1.hg38.bam': 20474056541,
 'gs://ccle_bams/hg38_wes/ACH-001711_3.hg38.bam': 21835106010,
 'gs://ccle_bams/hg38_wes/ACH-001847_1.hg38.bam': 19568519346,
 'gs://ccle_bams/hg38_wes/ACH-002021_1.hg38.bam': 13534304049,
 'gs://ccle_bams/hg38_wes/ACH-002048_1.hg38.bam': 18482553230,
 'gs://ccle_bams/hg38_wes/ACH-002065_1.hg38.bam': 14265095743,
 'gs://ccle_bams/hg38_wes/ACH-003000_1.hg38.bam': 13850429026,
 'gs://ccle_bams/rna/921.bam': 10416328884,
 'gs://ccle_bams/rna/94T778.bam': 15107396222,
 'gs://ccle_bams/rna/9505BIK/current/9505BIK.bam': 39504676493,
 'gs://ccle_bams/rna/95T1000.bam': 25114090709,
 'gs://ccle_bams/rna/A375SKINCJ2.bam': 4921397616,
 'gs://ccle_bams/rna/A388.bam': 6049094575,
 'gs://ccle_bams/rna/BPH1.bam': 6437868128,
 'gs://ccle_bams/rna/Becker.

## managing duplicates

In [93]:
names = {}
count = 0
dup= []
for k, val in sizes.items():
    if val in names:
        dup.append((k, names[val]))
        count+=1
    names[val] = k

In [94]:
dup

[('gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/94T778/v3/94T778.bam',
  'gs://ccle_bams/rna/94T778.bam'),
 ('gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/C396/v2/C396.bam',
  'gs://ccle_bams/rna/C396.bam'),
 ('gs://ccle_bams/rna/CCLP1.bam',
  'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/CCLP1/v1/CCLP1.bam'),
 ('gs://ccle_bams/rna/CII.bam',
  'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/CII/v3/CII.bam'),
 ('gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Aug2019/RP-1561/RNA/Becker/v1/Becker.bam',
  'gs://ccle_bams/rna/Becker.bam'),
 ('gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Batch2_July2019/RP-1561/RNA/CCLFUPGI0005T/v2/CCLFUPGI0005T.bam',
  'gs://ccle_bams/rna/CCLFUPGI0005T.bam'),
 ('gs://ccle_bams/rna/G415.bam',
  'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/G415/v1/G415.bam'),
 ('gs://ccle_bams/rna/GB2.bam',
  'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_July2019/RP-1561/RNA/GB2/v2/GB2.bam'),
 

In [30]:
count

161

In [41]:
rna = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq').get_samples().RNAseq_bam

APIException: Unable to query samples: (401) : <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head><script src="https://jsagent.tcell.io/tcellagent.min.js" tcellappid="FCProd-EBOgM" tcellapikey="AQQBBAFLGLOxL7VE9IF9ESlLvCxD5Ykr_7xkQKq_rgn_P58IWjOhOzIh6p3aI4pTWaprlUw" tcellbaseurl="https://us.agent.tcell.insight.rapid7.com/api/v1"></script>
<title>401 Unauthorized</title>
</head><body>
<h1>Unauthorized</h1>
<p>This server could not verify that you
are authorized to access the document
requested.  Either you supplied the wrong
credentials (e.g., bad password), or your
browser doesn't understand how to supply
the credentials required.</p>
<hr>
<address>Apache Server at api.firecloud.org Port 443</address>
</body></html>


In [145]:
keeprem = {} 
for i,j in dup:
    if ('rna' in i and 'wes' in j) or ('rna' in j and 'wes' in i):
        print('RNA/WES MISLABELLING')
        print(i,j)
    else:
        if '/v' in i:
            keeprem[j]=i
            if '/v' in j:
                print(i, j)
        elif '/v' in j:
            keeprem[i]=j
        else:
            print('NONE:')
            print(i,j)

gs://ccle_bams/rna/v2/ICC7.bam gs://ccle_bams/rna/v2.0/ICC7.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C817/EWS-502/current/EWS-502.bam gs://ccle_bams/rna/C817/EWS-502/current/EWS-502.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C817/EWS834/current/EWS834.bam gs://ccle_bams/rna/C817/EWS834/current/EWS834.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C817/TC-32/v3/TC-32.bam gs://ccle_bams/rna/C817/TC-32/v3/TC-32.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C817/TC-71/current/TC-71.bam gs://ccle_bams/rna/C817/TC-71/current/TC-71.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C817/TTC-466/current/TTC-466.bam gs://ccle_bams/rna/C817/TTC-466/current/TTC-466.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C818/CR_G-401/current/CR_G-401.bam gs://ccle_bams/rna/C818/CR_G-401/current/CR_G-401.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C836.K-562.bam gs://ccle_bams/rna/C836/K-562/v2/K-562.bam
RNA/WES MISLABELLING
gs://ccle_bams/wes/C836.KMS-26.bam gs://ccle_bams/rna/C836/KMS-26/v2/KMS-26.bam
RNA/WES MISLA

## managing the different versions

In [89]:
for k, val in sizes.items():
    sizes[k] = k.split('/')[-2]

In [ ]:
versions = set(sizes.values())
versions

In [ ]:
v1 = []
v2 = []
v3 = []
v4 = []
v5 = []
v6 = []
unknown = []
current= []
name = []
for k, val in sizes.items():
    name.append(k.split('/')[-1].split('.bam')[0])
    
    v1.append(k if val =='v1' else np.NaN)
    v2.append(k if val =='v2' else np.NaN)
    v3.append(k if val =='v3' else np.NaN)
    v4.append(k if val =='v4' else np.NaN)
    v5.append(k if val =='v5' else np.NaN)
    v6.append(k if val =='v6' else np.NaN)
    current.append(k if val =='current' else np.NaN)
    unknown.append(k if val in ['Getz_IBM_CellLines_Exomes_19samples_02012018', 'bam.analysis.ready', 'ccle.wes.gdc', 'wes', 'rna'] else np.NaN)

In [ ]:
data = pd.DataFrame({'unknown':unknown,
                    'v1':v1,
                    'v2':v2,
                    'v3':v3,
                    'v4':v4,
                    'v5':v5,
                    'v6':v6,
                    'current':current},index=name)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.columns

In [ ]:
len(data)

In [ ]:
gb = data.groupby(('index'))

In [ ]:
res = pd.DataFrame(gb[data.columns[0]].unique())

In [ ]:
for i in data.columns[1:]:
    res[i] = gb[i].unique()

In [ ]:
for k, val in res.iterrows():
    for j, i in val.iteritems():
        put=False
        for v in i:
            if v is not np.NaN:
                res.loc[k,j] = v
                put = True
        if not put:
            res.loc[k,j] = np.NaN

In [ ]:
res = res.drop('index',1)

In [ ]:
res.to_csv('state.csv')

In [ ]:
res = pd.read_csv('ccle_versions_state.csv')

In [ ]:
for i, val in res.iterrows():
    if val.v1 is np.nan:
        if val.unknown is not np.nan:
            res.loc[i,'v1'] = val.unknown
            res.loc[i,'unknown'] = np.nan
        elif val.current is not np.nan:
            res.loc[i,'v1'] = val.current
            res.loc[i,'current'] = np.nan
            

In [ ]:
res.isna().sum(0)/len(res)

In [ ]:
res

## IN RNA WORKSPACE BUT IN WES FOLDER

In [59]:
inRNAbutwesfolder={'gs://ccle_bams/wes/G41801.Caco-2.bam',
 'gs://ccle_bams/wes/G41809/NCI-H2077/current/NCI-H2077.bam',
 'gs://ccle_bams/wes/G27206.SNU-201.1.bam',
 'gs://ccle_bams/wes/G41763/U343/current/U343.bam',
 'gs://ccle_bams/wes/G41787.HEK_TE.bam',
 'gs://ccle_bams/wes/G41778.SLR20.bam',
 'gs://ccle_bams/wes/G41771.LN-319.bam',
 'gs://ccle_bams/wes/G28813.Hs_274.T.bam',
 'gs://ccle_bams/wes/G28860/Hs_751.T/current/Hs_751.T.bam',
 'gs://ccle_bams/wes/G41820/TM-87/current/TM-87.bam',
 'gs://ccle_bams/wes/G41760/U178/current/U178.bam',
 'gs://ccle_bams/wes/G41776/JH-ESOAD1/current/JH-ESOAD1.bam',
 'gs://ccle_bams/wes/G41777/SLR23/current/SLR23.bam',
 'gs://ccle_bams/wes/G35078.HDLM-2.bam',
 'gs://ccle_bams/wes/G41765/SLR24/current/SLR24.bam',
 'gs://ccle_bams/wes/G28885.Hs_852.T.3.bam',
 'gs://ccle_bams/wes/G41775/A1207/current/A1207.bam',
 'gs://ccle_bams/wes/G41796.HNT-34-2.bam',
 'gs://ccle_bams/wes/G41770.SLR25.bam',
 'gs://ccle_bams/wes/G27268.SK-HEP-1.1.bam',
 'gs://ccle_bams/wes/G35070/OVKATE/current/OVKATE.bam',
 'gs://ccle_bams/wes/G27506.RERF-LC-KJ.2.bam',
 'gs://ccle_bams/wes/G41759/SF767/current/SF767.bam',
 'gs://ccle_bams/wes/G41782/OELE/current/OELE.bam',
 'gs://ccle_bams/wes/G41789/EW8/current/EW8.bam',
 'gs://ccle_bams/wes/G35069.SNU-46.bam',
 'gs://ccle_bams/wes/G25203.NCI-H1092.1.bam',
 'gs://ccle_bams/wes/G41804.NCI-H2004_RT_2.bam',
 'gs://ccle_bams/wes/G41780/SLR26/current/SLR26.bam',
 'gs://ccle_bams/wes/G28037/KYO-1/v2/KYO-1.bam',
 'gs://ccle_bams/wes/G41784/HMEL/current/HMEL.bam',
 'gs://ccle_bams/wes/G41758/LN-443/current/LN-443.bam',
 'gs://ccle_bams/wes/G35068/SNU-283/current/SNU-283.bam',
 'gs://ccle_bams/wes/C1926/SNU-601/current/SNU-601.bam',
 'gs://ccle_bams/wes/G41813/A427/current/A427.bam',
 'gs://ccle_bams/wes/G41808/PC-9/current/PC-9.bam',
 'gs://ccle_bams/wes/G28857.HuH28.bam',
 'gs://ccle_bams/wes/G35076/NCI-H1373/current/NCI-H1373.bam',
 'gs://ccle_bams/wes/G41761/SF172/current/SF172.bam',
 'gs://ccle_bams/wes/G25205.NCI-H2286.1.bam',
 'gs://ccle_bams/wes/G130264/143B_1/current/143B_1.bam',
 'gs://ccle_bams/wes/G143832/CBAGPN/v1/CBAGPN.bam',
 'gs://ccle_bams/wes/G106825/CHLA266_1/current/CHLA266_1.bam',
 'gs://ccle_bams/wes/G106825/CHLA32_1/current/CHLA32_1.bam',
 'gs://ccle_bams/wes/G106825/CHLA57_1/current/CHLA57_1.bam',
 'gs://ccle_bams/wes/G106825/CHLA-9_1/current/CHLA-9_1.bam',
 'gs://ccle_bams/wes/G106825/CHLA99_1/current/CHLA99_1.bam',
 'gs://ccle_bams/wes/G130264/COV504_1/current/COV504_1.bam',
 'gs://ccle_bams/wes/G106825/CW9019_1/current/CW9019_1.bam',
 'gs://ccle_bams/wes/G102426/D425_1/current/D425_1.bam',
 'gs://ccle_bams/wes/G102426/D458_1/current/D458_1.bam',
 'gs://ccle_bams/wes/G143832/DL_/current/DL_.bam',
 'gs://ccle_bams/wes/G93756/DL-40_1/current/DL-40_1.bam',
 'gs://ccle_bams/wes/G106825/F5/current/F5.bam',
 'gs://ccle_bams/wes/G91718/FEPD_1/current/FEPD_1.bam',
 'gs://ccle_bams/wes/G28543.OCI-LY-19.1.bam',
 'gs://ccle_bams/wes/G91718/Karpas-384_1/current/Karpas-384_1.bam',
 'gs://ccle_bams/wes/G77557.KD_01.bam',
 'gs://ccle_bams/wes/G91718/KHYG-1_1/current/KHYG-1_1.bam',
 'gs://ccle_bams/wes/G131709/L82_1/current/L82_1.bam',
 'gs://ccle_bams/wes/G77556/MON_01/current/MON_01.bam',
 'gs://ccle_bams/wes/G26187/MONO-MAC-1/v3/MONO-MAC-1.bam',
 'gs://ccle_bams/wes/G93756/MTA_1/current/MTA_1.bam',
 'gs://ccle_bams/wes/G131709/My-La_1/current/My-La_1.bam',
 'gs://ccle_bams/wes/G91718/NKL_1/current/NKL_1.bam',
 'gs://ccle_bams/wes/G143832/OC316/current/OC316.bam',
 'gs://ccle_bams/wes/G102426/PEDS005PTFAD_1/current/PEDS005PTFAD_1.bam',
 'gs://ccle_bams/wes/G102426/R262_1/current/R262_1.bam',
 'gs://ccle_bams/wes/G136933/SCMCRM2_1/current/SCMCRM2_1.bam',
 'gs://ccle_bams/wes/G131767/SMZ-1_2/current/SMZ-1_2.bam',
 'gs://ccle_bams/wes/G18893/TC32/current/TC32.bam',
 'gs://ccle_bams/wes/G41786/TIG3-TD/current/TIG3-TD.bam',
 'gs://ccle_bams/wes/G77554/TTC-1240_01/current/TTC-1240_01.bam',
 'gs://ccle_bams/wes/G130264/127399_1/current/127399_1.bam',
 'gs://ccle_bams/wes/G106825/SW982_1/current/SW982_1.bam',
 'gs://ccle_bams/wes/G100662/BIN-67_1/current/BIN-67_1.bam',
 'gs://ccle_bams/wes/G106825/CHLA15_1/current/CHLA15_1.bam',
 'gs://ccle_bams/wes/G106825/COGN278_1/current/COGN278_1.bam',
 'gs://ccle_bams/wes/G106825/COGN305_1/current/COGN305_1.bam',
 'gs://ccle_bams/wes/G130264/NB1643_1/current/NB1643_1.bam',
 'gs://ccle_bams/wes/G143832/HA1E/v1/HA1E.bam',
 'gs://ccle_bams/wes/G130264/CME-1_1/current/CME-1_1.bam',
 'gs://ccle_bams/wes/G134624/A431_1/v4/A431_1.bam',
 'gs://ccle_bams/wes/G141708/C33A/current/C33A.bam',
 'gs://ccle_bams/wes/G134624/COLO794_1/current/COLO794_1.bam',
 'gs://ccle_bams/wes/G141708/COV413A/v2/COV413A.bam',
 'gs://ccle_bams/wes/G141708/GIMEN/current/GIMEN.bam',
 'gs://ccle_bams/wes/G141708/H103/v2/H103.bam',
 'gs://ccle_bams/wes/G141708/JOPACA1/current/JOPACA1.bam',
 'gs://ccle_bams/wes/G141708/LAN2/v2/LAN2.bam',
 'gs://ccle_bams/wes/G141708/MS751/v2/MS751.bam',
 'gs://ccle_bams/wes/G144159/NMB/v1/NMB.bam',
 'gs://ccle_bams/wes/G141708/OV17R/v2/OV17R.bam',
 'gs://ccle_bams/wes/G141708/PACADD119/current/PACADD119.bam',
 'gs://ccle_bams/wes/G141708/PACADD135/v2/PACADD135.bam',
 'gs://ccle_bams/wes/G144159/PACADD159/current/PACADD159.bam',
 'gs://ccle_bams/wes/G141708/PACADD165/v2/PACADD165.bam',
 'gs://ccle_bams/wes/G141708/PACADD188/current/PACADD188.bam',
 'gs://ccle_bams/wes/G134624/SCLC22H_1/current/SCLC22H_1.bam',
 'gs://ccle_bams/wes/G134624/SUM149PT_1/current/SUM149PT_1.bam',
 'gs://ccle_bams/wes/G134624/SUM159PT_1/v4/SUM159PT_1.bam',
 'gs://ccle_bams/wes/G134624/SUM299PE_1/current/SUM299PE_1.bam',
 'gs://ccle_bams/wes/G144963/SW954/v2/SW954.bam',
 'gs://ccle_bams/wes/G141708/UMUC16/current/UMUC16.bam',
 'gs://ccle_bams/wes/G141708/UMUC5/current/UMUC5.bam',
 'gs://ccle_bams/wes/G141708/UMUC10/current/UMUC10.bam',
 'gs://ccle_bams/wes/G141708/UMUC11/current/UMUC11.bam',
 'gs://ccle_bams/wes/G141708/UMUC6/v2/UMUC6.bam',
 'gs://ccle_bams/wes/G141708/UMUC7/v2/UMUC7.bam',
 'gs://ccle_bams/wes/G106825/TC138_1/current/TC138_1.bam',
 'gs://ccle_bams/wes/G106825/TC205_1/current/TC205_1.bam',
 'gs://ccle_bams/wes/G130264/PEDS092T__1/current/PEDS092T__1.bam',
 'gs://ccle_bams/wes/G134624/C10_1/v4/C10_1.bam',
 'gs://ccle_bams/wes/G134624/C75_1/v4/C75_1.bam',
 'gs://ccle_bams/wes/G134624/C80_1/v4/C80_1.bam',
 'gs://ccle_bams/wes/G134624/C99_1/current/C99_1.bam',
 'gs://ccle_bams/wes/G144159/EGI1/v1/EGI1.bam',
 'gs://ccle_bams/wes/G134624/ESO26_1/current/ESO26_1.bam',
 'gs://ccle_bams/wes/G134624/FLO1_1/current/FLO1_1.bam',
 'gs://ccle_bams/wes/G134624/HSC1_1/current/HSC1_1.bam',
 'gs://ccle_bams/wes/G144159/KKU100/v1/KKU100.bam',
 'gs://ccle_bams/wes/G144963/KMLS1/current/KMLS1.bam',
 'gs://ccle_bams/wes/G134624/MCC13_1/current/MCC13_1.bam',
 'gs://ccle_bams/wes/G134624/MCC26_1/current/MCC26_1.bam',
 'gs://ccle_bams/wes/G134624/MERO48A_1/v4/MERO48A_1.bam',
 'gs://ccle_bams/wes/G134624/MERO83_1/current/MERO83_1.bam',
 'gs://ccle_bams/wes/G134624/MERO84_1/current/MERO84_1.bam',
 'gs://ccle_bams/wes/G134624/MM386_1/v4/MM386_1.bam',
 'gs://ccle_bams/wes/G134624/MM426_1/v4/MM426_1.bam',
 'gs://ccle_bams/wes/G144159/NP5/v1/NP5.bam',
 'gs://ccle_bams/wes/G144159/ONDA9/v1/ONDA9.bam',
 'gs://ccle_bams/wes/G146853/PEO1/v2/PEO1.bam',
 'gs://ccle_bams/wes/G134624/SEKI_1/current/SEKI_1.bam',
 'gs://ccle_bams/wes/Getz_IBM_CellLines_Exomes_11samples_04122018/G141708/NA/SISO/v2/SISO.bam',
 'gs://ccle_bams/wes/G146853/SKGI/v2/SKGI.bam',
 'gs://ccle_bams/wes/G144963/U2904/current/U2904.bam',
 'gs://ccle_bams/wes/G144965/CAL72/v2/CAL72.bam',
 'gs://ccle_bams/wes/G116621/RHJT/current/RHJT.bam',
 'gs://ccle_bams/wes/G136933/RD_pcDNA1_1/v2/RD_pcDNA1_1.bam',
 'gs://ccle_bams/wes/G136933/RD_p3F__7_1/v2/RD_p3F__7_1.bam',
 'gs://ccle_bams/wes/G136933/RD_P3F__14_1/v2/RD_P3F__14_1.bam',
 'gs://ccle_bams/wes/G146344/ICC2/current/ICC2.bam'}

In [64]:
likely_wes= []
for val in inRNAbutwesfolder:
    res = os.popen('samtools view -H '+val).read()
    if 'bwa mem' in res:
        likely_wes.append(val)

In [65]:
likely_wes = ['gs://ccle_bams/wes/C1926/SNU-601/current/SNU-601.bam']

''

we found a likely wes file and a missing sample

In [57]:
inRNAbutwesfolder_name = [i.split('/')[-1] for i in inRNAbutwesfolder]

In [152]:
!gsutil ls gs://ccle_bams/wes/**U343*.bam

gs://ccle_bams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/U343/v1/U343.bam
gs://ccle_bams/wes/U343/current/U343.bam


In [160]:
bams = !gsutil ls gs://ccle_bams/rna/**/*SNU-601*.bam
set(bams) - set(likelymisslab)

{'gs://ccle_bams/rna/G27466.SNU-601.2.bam'}

In [ ]:
inRNAreplace =  
{"gs://ccle_bams/wes/G41763/U343/current/U343.bam":
"gs://ccle_bams/wes/U343/current/U343.bam",
"gs://ccle_bams/wes/C1926/SNU-601/current/SNU-601.bam":
"gs://ccle_bams/rna/G27466.SNU-601.2.bam"}

inWESreplace = 
{'gs://ccle_bams/wes/U343/current/U343.bam':
'gs://ccle_bams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/U343/v1/U343.bam'}

## finding additional mislabbeled

In [ ]:
a=5000
i=0
for k in names:
    i+=1
    if k in misclass or i<a:
        continue
    res = os.popen('samtools view -H '+k).read()
    if res == '':
        raise ValueError()
    if 'tophat' in res or 'STAR' in res:
        if '_bams/wes/' in k:
            misclass[k] = res
    elif 'bwa ' in res or 'maq' in res:
        if '_bams/rna/' in k:
            misclass[k] = res
    else:
        print res
        raise ValueError(k)
    print(str(i)+'\r')

In [53]:
likelymisslab= ['gs://ccle_bams/rna/G29750/HT-1376/v1/HT-1376.bam',
 'gs://ccle_bams/rna/G16640/BICR_16/current/BICR_16.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1734/current/NCI-H1734.bam',
 'gs://ccle_bams/rna/C1926/HCC2935/current/HCC2935.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1693/current/NCI-H1693.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1755/current/NCI-H1755.bam',
 'gs://ccle_bams/wes/G28054.KYSE-520.1.bam',
 'gs://ccle_bams/rna/G29750/huH-1/v1/huH-1.bam',
 'gs://ccle_bams/rna/C1926/SNU-283/current/SNU-283.bam',
 'gs://ccle_bams/rna/G16640/BFTC-909/current/BFTC-909.bam',
 'gs://ccle_bams/rna/G29750/JHOM-2B/v1/JHOM-2B.bam',
 'gs://ccle_bams/rna/G29750/NALM-6/v1/NALM-6.bam',
 'gs://ccle_bams/rna/G32831/HCC1599/v2/HCC1599.bam',
 'gs://ccle_bams/rna/G16640/SNU-1197/current/SNU-1197.bam',
 'gs://ccle_bams/rna/G16640/IST-MES2/current/IST-MES2.bam',
 'gs://ccle_bams/rna/G16640/Caki-2/current/Caki-2.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1838/current/NCI-H1838.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1092/v1/NCI-H1092.bam',
 'gs://ccle_bams/rna/G16640/SNU-489/current/SNU-489.bam',
 'gs://ccle_bams/wes/G136933/RD_pcDNA1_1/v2/RD_pcDNA1_1.bam',
 'gs://ccle_bams/rna/G29750/Hs_618.T/v2/Hs_618.T.bam',
 'gs://ccle_bams/wes/G134624/C10_1/v4/C10_1.bam',
 'gs://ccle_bams/rna/G16640/HCC1143/current/HCC1143.bam',
 'gs://ccle_bams/rna/G16640/TE-9/current/TE-9.bam',
 'gs://ccle_bams/rna/G16640/RL/current/RL.bam',
 'gs://ccle_bams/wes/G77554/TTC-1240_01/current/TTC-1240_01.bam',
 'gs://ccle_bams/rna/G29750/SW837/v1/SW837.bam',
 'gs://ccle_bams/rna/G16640/LCLC-103H/current/LCLC-103H.bam',
 'gs://ccle_bams/rna/G16640/PK-45H/current/PK-45H.bam',
 'gs://ccle_bams/rna/G16640/Hs_766T/current/Hs_766T.bam',
 'gs://ccle_bams/rna/G16640/NCI-H211/current/NCI-H211.bam',
 'gs://ccle_bams/rna/G29750/WM-115/v3/WM-115.bam',
 'gs://ccle_bams/rna/G16384/NCI-H2196/v1/NCI-H2196.bam',
 'gs://ccle_bams/rna/G16640/LS513/current/LS513.bam',
 'gs://ccle_bams/rna/G29750/COLO-800/v2/COLO-800.bam',
 'gs://ccle_bams/rna/C1926/CJM/current/CJM.bam',
 'gs://ccle_bams/rna/C1926/HCC-2279/current/HCC-2279.bam',
 'gs://ccle_bams/rna/G16640/Panc_05.04/current/Panc_05.04.bam',
 'gs://ccle_bams/rna/G32831/NCI-H838/v1/NCI-H838.bam',
 'gs://ccle_bams/rna/G29750/Hs_751.T/v2/Hs_751.T.bam',
 'gs://ccle_bams/rna/G16640/Panc_02.13/current/Panc_02.13.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1105/current/NCI-H1105.bam',
 'gs://ccle_bams/rna/G29750/SNU-761/v1/SNU-761.bam',
 'gs://ccle_bams/rna/G16640/KMS-18/current/KMS-18.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1666/v1/NCI-H1666.bam',
 'gs://ccle_bams/rna/G16640/TF-1/current/TF-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1048/v1/NCI-H1048.bam',
 'gs://ccle_bams/rna/G16640/Loucy/current/Loucy.bam',
 'gs://ccle_bams/rna/G29750/HEC-265/v3/HEC-265.bam',
 'gs://ccle_bams/rna/G16377/KYSE-410/v2/KYSE-410.bam',
 'gs://ccle_bams/rna/G29750/U-87_MG/v1/U-87_MG.bam',
 'gs://ccle_bams/rna/G16441/DB/v1/DB.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2087/v3/NCI-H2087.bam',
 'gs://ccle_bams/rna/G29750/Kasumi-6/v4/Kasumi-6.bam',
 'gs://ccle_bams/rna/G29750/KMS-34/v1/KMS-34.bam',
 'gs://ccle_bams/rna/G16640/SKM-1/current/SKM-1.bam',
 'gs://ccle_bams/rna/C817/EWS834/current/EWS834.bam',
 'gs://ccle_bams/rna/G16640/SK-MEL-2/current/SK-MEL-2.bam',
 'gs://ccle_bams/wes/G41780/SLR26/current/SLR26.bam',
 'gs://ccle_bams/wes/G41789/EW8/current/EW8.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1792/current/NCI-H1792.bam',
 'gs://ccle_bams/rna/G16640/MPP_89/current/MPP_89.bam',
 'gs://ccle_bams/rna/G29750/HuCCT1/v2/HuCCT1.bam',
 'gs://ccle_bams/rna/G16640/JHH-1/current/JHH-1.bam',
 'gs://ccle_bams/rna/G16640/COR-L279/current/COR-L279.bam',
 'gs://ccle_bams/wes/G28860/Hs_751.T/current/Hs_751.T.bam',
 'gs://ccle_bams/rna/G32831/NCI-H1975/v2/NCI-H1975.bam',
 'gs://ccle_bams/rna/G29750/MONO-MAC-1/v1/MONO-MAC-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2227/v1/NCI-H2227.bam',
 'gs://ccle_bams/rna/G16640/TE_617.T/current/TE_617.T.bam',
 'gs://ccle_bams/wes/G91718/Karpas-384_1/current/Karpas-384_1.bam',
 'gs://ccle_bams/rna/G16640/DND-41/current/DND-41.bam',
 'gs://ccle_bams/rna/G16640/RI-1/current/RI-1.bam',
 'gs://ccle_bams/rna/G29750/RH-41/v3/RH-41.bam',
 'gs://ccle_bams/rna/G16640/MEG-01/current/MEG-01.bam',
 'gs://ccle_bams/rna/G16640/CMK/current/CMK.bam',
 'gs://ccle_bams/rna/G29750/MDA-MB-361/v1/MDA-MB-361.bam',
 'gs://ccle_bams/rna/G29750/Hs_616.T/v3/Hs_616.T.bam',
 'gs://ccle_bams/wes/G35069.SNU-46.bam',
 'gs://ccle_bams/rna/G29750/59M/v3/59M.bam',
 'gs://ccle_bams/rna/G32831/OV56/v2/OV56.bam',
 'gs://ccle_bams/rna/G16382/RERF-LC-MS/v2/RERF-LC-MS.bam',
 'gs://ccle_bams/rna/G16640/Hs_939.T/current/Hs_939.T.bam',
 'gs://ccle_bams/rna/G16350/DMS_273/v4/DMS_273.bam',
 'gs://ccle_bams/wes/G41808/PC-9/current/PC-9.bam',
 'gs://ccle_bams/rna/G29750/BV-173/v1/BV-173.bam',
 'gs://ccle_bams/rna/G29750/JHUEM-7/v2/JHUEM-7.bam',
 'gs://ccle_bams/rna/G32831/JHUEM-1/v2/JHUEM-1.bam',
 'gs://ccle_bams/rna/G32831/ES-2/v2/ES-2.bam',
 'gs://ccle_bams/rna/G29750/OE21/v3/OE21.bam',
 'gs://ccle_bams/rna/G16640/IGR-37/current/IGR-37.bam',
 'gs://ccle_bams/wes/G41782/OELE/current/OELE.bam',
 'gs://ccle_bams/rna/G29750/KYSE-150/v1/KYSE-150.bam',
 'gs://ccle_bams/rna/G16640/SW_1783/current/SW_1783.bam',
 'gs://ccle_bams/rna/G16640/HT-29/current/HT-29.bam',
 'gs://ccle_bams/rna/G29750/YD-8/v3/YD-8.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1915/current/NCI-H1915.bam',
 'gs://ccle_bams/rna/C1926/SW480/current/SW480.bam',
 'gs://ccle_bams/rna/G16640/HT55/current/HT55.bam',
 'gs://ccle_bams/rna/G16640/U266B1/current/U266B1.bam',
 'gs://ccle_bams/rna/G29750/L-540/v1/L-540.bam',
 'gs://ccle_bams/rna/G16640/EFE-184/current/EFE-184.bam',
 'gs://ccle_bams/rna/G16640/OVTOKO/current/OVTOKO.bam',
 'gs://ccle_bams/rna/G29750/HMCB/v1/HMCB.bam',
 'gs://ccle_bams/rna/G16640/Daudi/current/Daudi.bam',
 'gs://ccle_bams/rna/G16640/RERF-LC-MS/current/RERF-LC-MS.bam',
 'gs://ccle_bams/rna/G16640/NCI-H322/current/NCI-H322.bam',
 'gs://ccle_bams/rna/G29750/LC-1F/v1/LC-1F.bam',
 'gs://ccle_bams/rna/G29750/Caki-2/v4/Caki-2.bam',
 'gs://ccle_bams/rna/G29750/BHT-101/v2/BHT-101.bam',
 'gs://ccle_bams/wes/G131709/L82_1/current/L82_1.bam',
 'gs://ccle_bams/rna/G16640/Li-7/current/Li-7.bam',
 'gs://ccle_bams/rna/G29750/KP-3/v1/KP-3.bam',
 'gs://ccle_bams/rna/G16640/PA-TU-8988S/current/PA-TU-8988S.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1975/v3/NCI-H1975.bam',
 'gs://ccle_bams/rna/G29750/J82/v2/J82.bam',
 'gs://ccle_bams/rna/G29750/8-MG-BA/v1/8-MG-BA.bam',
 'gs://ccle_bams/rna/G29750/SNU-423/v2/SNU-423.bam',
 'gs://ccle_bams/rna/C1926/SNU-1040/current/SNU-1040.bam',
 'gs://ccle_bams/wes/G27492.SNU-719.2.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2286/v2/NCI-H2286.bam',
 'gs://ccle_bams/rna/G16640/A-673/current/A-673.bam',
 'gs://ccle_bams/rna/G29750/TF-1/v2/TF-1.bam',
 'gs://ccle_bams/rna/G16640/SK-N-DZ/current/SK-N-DZ.bam',
 'gs://ccle_bams/rna/G29750/LCLC-103H/v1/LCLC-103H.bam',
 'gs://ccle_bams/rna/C1926/SNU-1066/current/SNU-1066.bam',
 'gs://ccle_bams/rna/G29750/CFPAC-1/v1/CFPAC-1.bam',
 'gs://ccle_bams/rna/G16640/CADO-ES1/current/CADO-ES1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H460/v2/NCI-H460.bam',
 'gs://ccle_bams/wes/G27275.TE-9.1.bam',
 'gs://ccle_bams/rna/G16640/SU-DHL-6/current/SU-DHL-6.bam',
 'gs://ccle_bams/rna/G16640/Hs_281.T/current/Hs_281.T.bam',
 'gs://ccle_bams/rna/G29750/Loucy/v1/Loucy.bam',
 'gs://ccle_bams/wes/G41758/LN-443/current/LN-443.bam',
 'gs://ccle_bams/rna/G16640/SNU-C5/current/SNU-C5.bam',
 'gs://ccle_bams/rna/G29750/GCT/v1/GCT.bam',
 'gs://ccle_bams/rna/G29750/SF268/v2/SF268.bam',
 'gs://ccle_bams/rna/G29750/IPC-298/v1/IPC-298.bam',
 'gs://ccle_bams/rna/G29750/NCI-H747/v2/NCI-H747.bam',
 'gs://ccle_bams/rna/G29750/ONCO-DG-1/v2/ONCO-DG-1.bam',
 'gs://ccle_bams/rna/G16450/NUGC-3/v1/NUGC-3.bam',
 'gs://ccle_bams/rna/G29750/NCI-H69/v1/NCI-H69.bam',
 'gs://ccle_bams/rna/G16640/Pfeiffer/current/Pfeiffer.bam',
 'gs://ccle_bams/rna/G16640/G-361/current/G-361.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1355/v3/NCI-H1355.bam',
 'gs://ccle_bams/rna/G16640/253J/current/253J.bam',
 'gs://ccle_bams/wes/G30554.KASUMI-1.1.bam',
 'gs://ccle_bams/rna/G16640/SNU-878/current/SNU-878.bam',
 'gs://ccle_bams/wes/G35078.HDLM-2.bam',
 'gs://ccle_bams/rna/G16420/MFE-296/v1/MFE-296.bam',
 'gs://ccle_bams/rna/C1926/LU65/current/LU65.bam',
 'gs://ccle_bams/rna/G16640/Mino/current/Mino.bam',
 'gs://ccle_bams/rna/G16640/HEC-108/current/HEC-108.bam',
 'gs://ccle_bams/rna/G29750/COLO-783/v4/COLO-783.bam',
 'gs://ccle_bams/rna/G16640/Hs_742.T/current/Hs_742.T.bam',
 'gs://ccle_bams/wes/G106825/CHLA99_1/current/CHLA99_1.bam',
 'gs://ccle_bams/rna/G32831/HPAC/v2/HPAC.bam',
 'gs://ccle_bams/rna/G16485/T24/v1/T24.bam',
 'gs://ccle_bams/rna/G29750/SNU-308/v2/SNU-308.bam',
 'gs://ccle_bams/wes/G116621/RHJT/current/RHJT.bam',
 'gs://ccle_bams/wes/G136933/RD_p3F__7_1/v2/RD_p3F__7_1.bam',
 'gs://ccle_bams/rna/G16640/SNU-1196/current/SNU-1196.bam',
 'gs://ccle_bams/rna/G29750/MOLT-16/v1/MOLT-16.bam',
 'gs://ccle_bams/rna/C1926/HCC-1588/current/HCC-1588.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1385/current/NCI-H1385.bam',
 'gs://ccle_bams/rna/G16640/ESS-1/current/ESS-1.bam',
 'gs://ccle_bams/wes/G141708/UMUC5/current/UMUC5.bam',
 'gs://ccle_bams/rna/G16640/GMS-10/current/GMS-10.bam',
 'gs://ccle_bams/rna/G29750/OAW42/v2/OAW42.bam',
 'gs://ccle_bams/rna/G29750/IST-MES2/v3/IST-MES2.bam',
 'gs://ccle_bams/rna/G16640/DMS_53/current/DMS_53.bam',
 'gs://ccle_bams/rna/G32831/NCI-H358/v1/NCI-H358.bam',
 'gs://ccle_bams/rna/G29750/COR-L88/v2/COR-L88.bam',
 'gs://ccle_bams/rna/G16640/L-363/current/L-363.bam',
 'gs://ccle_bams/wes/G134624/FLO1_1/current/FLO1_1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2081/v3/NCI-H2081.bam',
 'gs://ccle_bams/wes/G26259.AML-193.2.bam',
 'gs://ccle_bams/rna/G32831/RMG-I/v2/RMG-I.bam',
 'gs://ccle_bams/rna/G16640/Hs_852.T/current/Hs_852.T.bam',
 'gs://ccle_bams/rna/G16640/HCC-78/current/HCC-78.bam',
 'gs://ccle_bams/rna/G32831/NCI-H1703/v3/NCI-H1703.bam',
 'gs://ccle_bams/rna/G29750/CMK/v4/CMK.bam',
 'gs://ccle_bams/rna/G16640/MKN74/current/MKN74.bam',
 'gs://ccle_bams/rna/G16365/HEL_92.1.7/v1/HEL_92.1.7.bam',
 'gs://ccle_bams/rna/G29750/HCC4006/v3/HCC4006.bam',
 'gs://ccle_bams/rna/G16640/SU-DHL-1/current/SU-DHL-1.bam',
 'gs://ccle_bams/rna/G29750/NB-1/v2/NB-1.bam',
 'gs://ccle_bams/wes/G106825/SW982_1/current/SW982_1.bam',
 'gs://ccle_bams/rna/G16640/HARA/current/HARA.bam',
 'gs://ccle_bams/rna/G16640/RVH-421/current/RVH-421.bam',
 'gs://ccle_bams/rna/G16640/HCC70/current/HCC70.bam',
 'gs://ccle_bams/wes/G91718/OCILY12_1/current/OCILY12_1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H358/current/NCI-H358.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1623/current/NCI-H1623.bam',
 'gs://ccle_bams/rna/G16640/Reh/current/Reh.bam',
 'gs://ccle_bams/rna/G29750/U-118_MG/v2/U-118_MG.bam',
 'gs://ccle_bams/rna/G16362/23132_87/v2/23132_87.bam',
 'gs://ccle_bams/rna/G29750/253J-BV/v1/253J-BV.bam',
 'gs://ccle_bams/rna/G16640/CAL-85-1/current/CAL-85-1.bam',
 'gs://ccle_bams/wes/G41803/HeLa/current/HeLa.bam',
 'gs://ccle_bams/rna/C1926/SW403/current/SW403.bam',
 'gs://ccle_bams/rna/G16640/JK-1/current/JK-1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1930/current/NCI-H1930.bam',
 'gs://ccle_bams/rna/G16640/UACC-893/current/UACC-893.bam',
 'gs://ccle_bams/rna/G16640/NCI-H69/current/NCI-H69.bam',
 'gs://ccle_bams/rna/G32831/NCI-H460/v2/NCI-H460.bam',
 'gs://ccle_bams/rna/G29750/Hs_888.T/v1/Hs_888.T.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2106/current/NCI-H2106.bam',
 'gs://ccle_bams/rna/G32831/KYM-1/v2/KYM-1.bam',
 'gs://ccle_bams/rna/C1926/KE-39/current/KE-39.bam',
 'gs://ccle_bams/rna/G29750/JHOS-2/v4/JHOS-2.bam',
 'gs://ccle_bams/rna/G29750/SK-N-SH/v2/SK-N-SH.bam',
 'gs://ccle_bams/rna/G16640/AML-193/current/AML-193.bam',
 'gs://ccle_bams/rna/G16401/NCI-H1876/v1/NCI-H1876.bam',
 'gs://ccle_bams/rna/G16640/MONO-MAC-6/current/MONO-MAC-6.bam',
 'gs://ccle_bams/rna/G16460/NCI-N87/v1/NCI-N87.bam',
 'gs://ccle_bams/rna/G29750/SNB75/v2/SNB75.bam',
 'gs://ccle_bams/rna/G16640/HT-144/current/HT-144.bam',
 'gs://ccle_bams/rna/G29750/PA-TU-8988S/v2/PA-TU-8988S.bam',
 'gs://ccle_bams/rna/G16482/FaDu/v1/FaDu.bam',
 'gs://ccle_bams/rna/G29750/MDA-MB-157/v2/MDA-MB-157.bam',
 'gs://ccle_bams/wes/G106825/F5/current/F5.bam',
 'gs://ccle_bams/rna/G16640/Hs_870.T/current/Hs_870.T.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1339/v2/NCI-H1339.bam',
 'gs://ccle_bams/rna/G32831/LAMA-84/v1/LAMA-84.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2009/current/NCI-H2009.bam',
 'gs://ccle_bams/rna/G29750/A101D/v1/A101D.bam',
 'gs://ccle_bams/rna/G16640/Panc_08.13/current/Panc_08.13.bam',
 'gs://ccle_bams/rna/G29750/NCO2/v2/NCO2.bam',
 'gs://ccle_bams/rna/G29750/EFO-27/v3/EFO-27.bam',
 'gs://ccle_bams/rna/C1926/NCI-H854/current/NCI-H854.bam',
 'gs://ccle_bams/rna/C1926/SNU-216/current/SNU-216.bam',
 'gs://ccle_bams/wes/G106825/TC205_1/current/TC205_1.bam',
 'gs://ccle_bams/rna/G16640/DMS_273/current/DMS_273.bam',
 'gs://ccle_bams/rna/G16388/NCI-H2081/v1/NCI-H2081.bam',
 'gs://ccle_bams/rna/G32831/SW_1783/v1/SW_1783.bam',
 'gs://ccle_bams/rna/G16640/JJN-3/current/JJN-3.bam',
 'gs://ccle_bams/rna/G32831/SNU-182/v1/SNU-182.bam',
 'gs://ccle_bams/rna/G29750/NCI-H196/v2/NCI-H196.bam',
 'gs://ccle_bams/rna/G16640/Calu-6/current/Calu-6.bam',
 'gs://ccle_bams/rna/G16404/NCI-H1650/v1/NCI-H1650.bam',
 'gs://ccle_bams/rna/G29750/HCC1187/v2/HCC1187.bam',
 'gs://ccle_bams/wes/G146344/ICC2/current/ICC2.bam',
 'gs://ccle_bams/wes/G28824.HEC-265.3.bam',
 'gs://ccle_bams/rna/G29750/OVCAR-8/v1/OVCAR-8.bam',
 'gs://ccle_bams/rna/C1926/NCI-H23/current/NCI-H23.bam',
 'gs://ccle_bams/rna/G16640/MJ/current/MJ.bam',
 'gs://ccle_bams/rna/G29750/KURAMOCHI/v1/KURAMOCHI.bam',
 'gs://ccle_bams/rna/G29750/TE-5/v1/TE-5.bam',
 'gs://ccle_bams/rna/G16640/MDST8/current/MDST8.bam',
 'gs://ccle_bams/rna/G16640/RERF-GC-1B/current/RERF-GC-1B.bam',
 'gs://ccle_bams/rna/G29750/HCC202/v1/HCC202.bam',
 'gs://ccle_bams/rna/G32831/OAW42/v2/OAW42.bam',
 'gs://ccle_bams/rna/G16640/NCI-H226/current/NCI-H226.bam',
 'gs://ccle_bams/rna/G29750/YKG1/v1/YKG1.bam',
 'gs://ccle_bams/rna/G29750/SW1463/v2/SW1463.bam',
 'gs://ccle_bams/rna/G16640/Ki-JK/current/Ki-JK.bam',
 'gs://ccle_bams/rna/G29750/COV504/v2/COV504.bam',
 'gs://ccle_bams/rna/G16640/SEM/current/SEM.bam',
 'gs://ccle_bams/rna/G29750/SW_1573/v2/SW_1573.bam',
 'gs://ccle_bams/rna/G16440/MKN74/v1/MKN74.bam',
 'gs://ccle_bams/rna/G29750/HPB-ALL/v2/HPB-ALL.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1869/v3/NCI-H1869.bam',
 'gs://ccle_bams/rna/G16640/COLO_741/current/COLO_741.bam',
 'gs://ccle_bams/wes/G134624/SUM159PT_1/v4/SUM159PT_1.bam',
 'gs://ccle_bams/rna/C1926/Hs_737.T/current/Hs_737.T.bam',
 'gs://ccle_bams/rna/G29750/OE19/v2/OE19.bam',
 'gs://ccle_bams/rna/C817/TC-32/v3/TC-32.bam',
 'gs://ccle_bams/wes/G141708/COV413A/v2/COV413A.bam',
 'gs://ccle_bams/rna/C1926/ECC10/current/ECC10.bam',
 'gs://ccle_bams/rna/G29750/8505C/v1/8505C.bam',
 'gs://ccle_bams/rna/C1926/MHH-ES-1/current/MHH-ES-1.bam',
 'gs://ccle_bams/wes/G144965/CAL72/v2/CAL72.bam',
 'gs://ccle_bams/rna/G29750/HEC-151/v4/HEC-151.bam',
 'gs://ccle_bams/rna/G29750/MDST8/v2/MDST8.bam',
 'gs://ccle_bams/rna/G16640/IGR-39/current/IGR-39.bam',
 'gs://ccle_bams/rna/G29750/SW_780/v1/SW_780.bam',
 'gs://ccle_bams/rna/G32831/AN3_CA/v2/AN3_CA.bam',
 'gs://ccle_bams/rna/G16640/Hs_578T/current/Hs_578T.bam',
 'gs://ccle_bams/rna/C1926/IA-LM/current/IA-LM.bam',
 'gs://ccle_bams/rna/G29750/SK-MES-1/v2/SK-MES-1.bam',
 'gs://ccle_bams/wes/G141708/CASKI/v3/CASKI.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2291/current/NCI-H2291.bam',
 'gs://ccle_bams/rna/G16640/NCI-H727/current/NCI-H727.bam',
 'gs://ccle_bams/rna/G16640/DMS_114/current/DMS_114.bam',
 'gs://ccle_bams/rna/G16640/BT-474/current/BT-474.bam',
 'gs://ccle_bams/rna/G16640/MOLT-13/current/MOLT-13.bam',
 'gs://ccle_bams/rna/G16640/SK-MEL-24/current/SK-MEL-24.bam',
 'gs://ccle_bams/rna/G16640/ONS-76/current/ONS-76.bam',
 'gs://ccle_bams/rna/G29750/KYSE-410/v1/KYSE-410.bam',
 'gs://ccle_bams/rna/G29750/KS-1/v1/KS-1.bam',
 'gs://ccle_bams/rna/G29750/CAS-1/v1/CAS-1.bam',
 'gs://ccle_bams/rna/G32831/CAL-51/v1/CAL-51.bam',
 'gs://ccle_bams/rna/G16640/OCI-AML5/current/OCI-AML5.bam',
 'gs://ccle_bams/wes/G144963/U2904/current/U2904.bam',
 'gs://ccle_bams/rna/G16640/SW48/current/SW48.bam',
 'gs://ccle_bams/rna/G29750/KM12/v1/KM12.bam',
 'gs://ccle_bams/rna/G16640/WSU-DLCL2/current/WSU-DLCL2.bam',
 'gs://ccle_bams/rna/G29750/COLO-320/v2/COLO-320.bam',
 'gs://ccle_bams/rna/G29750/PSN1/v1/PSN1.bam',
 'gs://ccle_bams/rna/G16640/Detroit_562/current/Detroit_562.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2286/current/NCI-H2286.bam',
 'gs://ccle_bams/rna/G29750/HMC-1-8/v2/HMC-1-8.bam',
 'gs://ccle_bams/rna/G29750/HLF/v2/HLF.bam',
 'gs://ccle_bams/wes/G144159/NP5/v1/NP5.bam',
 'gs://ccle_bams/rna/G29750/KYSE-30/v2/KYSE-30.bam',
 'gs://ccle_bams/rna/G16640/GAMG/current/GAMG.bam',
 'gs://ccle_bams/rna/G16425/HCT-15/v1/HCT-15.bam',
 'gs://ccle_bams/rna/G32831/HT-1376/v2/HT-1376.bam',
 'gs://ccle_bams/rna/G16640/Panc_03.27/current/Panc_03.27.bam',
 'gs://ccle_bams/rna/G32831/MFE-319/v2/MFE-319.bam',
 'gs://ccle_bams/rna/C1926/ACC-MESO-1/current/ACC-MESO-1.bam',
 'gs://ccle_bams/rna/G16640/KURAMOCHI/current/KURAMOCHI.bam',
 'gs://ccle_bams/rna/G29750/BxPC-3/v1/BxPC-3.bam',
 'gs://ccle_bams/rna/G29750/KMS-11/v3/KMS-11.bam',
 'gs://ccle_bams/rna/G16438/5637/v1/5637.bam',
 'gs://ccle_bams/rna/G32831/ZR-75-1/v1/ZR-75-1.bam',
 'gs://ccle_bams/wes/G41768/SLR21/current/SLR21.bam',
 'gs://ccle_bams/rna/G32831/LN-18/v2/LN-18.bam',
 'gs://ccle_bams/rna/G29750/OVTOKO/v5/OVTOKO.bam',
 'gs://ccle_bams/rna/G16640/TUHR14TKB/current/TUHR14TKB.bam',
 'gs://ccle_bams/wes/G106825/CHLA15_1/current/CHLA15_1.bam',
 'gs://ccle_bams/wes/G28885.Hs_852.T.3.bam',
 'gs://ccle_bams/rna/G29750/SNU-387/v3/SNU-387.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1581/current/NCI-H1581.bam',
 'gs://ccle_bams/rna/G29750/MDA-MB-231/v2/MDA-MB-231.bam',
 'gs://ccle_bams/rna/G16640/MOLM-6/current/MOLM-6.bam',
 'gs://ccle_bams/wes/G41761/SF172/current/SF172.bam',
 'gs://ccle_bams/rna/G16640/CA46/current/CA46.bam',
 'gs://ccle_bams/rna/G29750/JHH-4/v1/JHH-4.bam',
 'gs://ccle_bams/rna/G16640/MHH-CALL-4/current/MHH-CALL-4.bam',
 'gs://ccle_bams/rna/C1926/NCI-H727/current/NCI-H727.bam',
 'gs://ccle_bams/rna/G16640/U-BLC1/current/U-BLC1.bam',
 'gs://ccle_bams/rna/G16640/SU.86.86/current/SU.86.86.bam',
 'gs://ccle_bams/rna/G29750/TYK-nu/v2/TYK-nu.bam',
 'gs://ccle_bams/rna/G32831/HCC1428/v1/HCC1428.bam',
 'gs://ccle_bams/wes/G143832/CBAGPN/v1/CBAGPN.bam',
 'gs://ccle_bams/rna/G16640/HCC827/current/HCC827.bam',
 'gs://ccle_bams/wes/G141708/OV17R/v2/OV17R.bam',
 'gs://ccle_bams/rna/G32831/COLO_829/v1/COLO_829.bam',
 'gs://ccle_bams/rna/G16640/SK-N-FI/current/SK-N-FI.bam',
 'gs://ccle_bams/wes/G146853/PEO1/v2/PEO1.bam',
 'gs://ccle_bams/rna/G16640/SNU-423/current/SNU-423.bam',
 'gs://ccle_bams/rna/G48942/PK-1/current/PK-1.bam',
 'gs://ccle_bams/rna/C1926/SNU-61/current/SNU-61.bam',
 'gs://ccle_bams/rna/G32831/MFE-280/v1/MFE-280.bam',
 'gs://ccle_bams/wes/RP-1561/Exome/NOZ/current/NOZ.bam',
 'gs://ccle_bams/rna/G16437/SK-MEL-30/v1/SK-MEL-30.bam',
 'gs://ccle_bams/rna/G29750/SNU-201/v4/SNU-201.bam',
 'gs://ccle_bams/rna/G16640/Caov-3/current/Caov-3.bam',
 'gs://ccle_bams/rna/G29750/NCI-H889/v2/NCI-H889.bam',
 'gs://ccle_bams/rna/C1926/SNU-175/current/SNU-175.bam',
 'gs://ccle_bams/wes/G141708/H103/v2/H103.bam',
 'gs://ccle_bams/rna/G32831/G-402/v2/G-402.bam',
 'gs://ccle_bams/rna/G29750/BICR_18/v1/BICR_18.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1648/v1/NCI-H1648.bam',
 'gs://ccle_bams/wes/G41716.IA-LM.5.bam',
 'gs://ccle_bams/rna/G16640/Kasumi-6/current/Kasumi-6.bam',
 'gs://ccle_bams/rna/G29750/COLO_668/v5/COLO_668.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1563/current/NCI-H1563.bam',
 'gs://ccle_bams/rna/G29750/SNU-685/v3/SNU-685.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1395/current/NCI-H1395.bam',
 'gs://ccle_bams/rna/G29750/LS513/v2/LS513.bam',
 'gs://ccle_bams/rna/G29750/DBTRG-05MG/v1/DBTRG-05MG.bam',
 'gs://ccle_bams/rna/C1926/SNU-5/current/SNU-5.bam',
 'gs://ccle_bams/rna/G29750/Hs_604.T/v3/Hs_604.T.bam',
 'gs://ccle_bams/rna/G29750/RPMI-8402/v2/RPMI-8402.bam',
 'gs://ccle_bams/rna/G29750/SNU-478/v2/SNU-478.bam',
 'gs://ccle_bams/rna/G16640/PA-TU-8988T/current/PA-TU-8988T.bam',
 'gs://ccle_bams/rna/G29750/HEC-108/v3/HEC-108.bam',
 'gs://ccle_bams/rna/G16358/NCI-H1184/v1/NCI-H1184.bam',
 'gs://ccle_bams/wes/G134624/C80_1/v4/C80_1.bam',
 'gs://ccle_bams/rna/C1926/KYO-1/current/KYO-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H292/v2/NCI-H292.bam',
 'gs://ccle_bams/rna/G29750/UACC-62/v2/UACC-62.bam',
 'gs://ccle_bams/rna/G16640/NCI-H660/current/NCI-H660.bam',
 'gs://ccle_bams/rna/G29750/Hs_839.T/v2/Hs_839.T.bam',
 'gs://ccle_bams/rna/G16640/HEC-6/current/HEC-6.bam',
 'gs://ccle_bams/rna/G16640/SCaBER/current/SCaBER.bam',
 'gs://ccle_bams/rna/G29750/CCF-STTG1/v3/CCF-STTG1.bam',
 'gs://ccle_bams/rna/G16640/L-1236/current/L-1236.bam',
 'gs://ccle_bams/rna/G16640/SK-N-MC/current/SK-N-MC.bam',
 'gs://ccle_bams/rna/G29750/HT-144/v1/HT-144.bam',
 'gs://ccle_bams/rna/G29750/ACC-MESO-1/v2/ACC-MESO-1.bam',
 'gs://ccle_bams/rna/G29750/DMS_79/v2/DMS_79.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2066/v1/NCI-H2066.bam',
 'gs://ccle_bams/wes/G106825/CHLA32_1/current/CHLA32_1.bam',
 'gs://ccle_bams/rna/G16640/SCC-25/current/SCC-25.bam',
 'gs://ccle_bams/rna/G29750/Hs_606.T/v3/Hs_606.T.bam',
 'gs://ccle_bams/rna/G32831/HCC1937/v2/HCC1937.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2122/current/NCI-H2122.bam',
 'gs://ccle_bams/rna/G16640/647-V/current/647-V.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1963/current/NCI-H1963.bam',
 'gs://ccle_bams/rna/G29750/EHEB/v1/EHEB.bam',
 'gs://ccle_bams/rna/G16640/SNU-1/current/SNU-1.bam',
 'gs://ccle_bams/rna/G16640/769-P/current/769-P.bam',
 'gs://ccle_bams/rna/G16640/SW_1353/current/SW_1353.bam',
 'gs://ccle_bams/rna/G29750/TE-10/v1/TE-10.bam',
 'gs://ccle_bams/rna/G16640/SR-786/current/SR-786.bam',
 'gs://ccle_bams/rna/G29750/A2058/v2/A2058.bam',
 'gs://ccle_bams/rna/C836/KMS-26/v2/KMS-26.bam',
 'gs://ccle_bams/rna/G29750/SNU-1041/v3/SNU-1041.bam',
 'gs://ccle_bams/rna/G29750/FU-OV-1/v1/FU-OV-1.bam',
 'gs://ccle_bams/rna/G16640/KMS-26/current/KMS-26.bam',
 'gs://ccle_bams/wes/G27206.SNU-201.1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1299/current/NCI-H1299.bam',
 'gs://ccle_bams/wes/G28543.OCI-LY-19.1.bam',
 'gs://ccle_bams/rna/G16640/KS-1/current/KS-1.bam',
 'gs://ccle_bams/rna/G32831/SNU-119/v2/SNU-119.bam',
 'gs://ccle_bams/rna/G16640/PANC-1/current/PANC-1.bam',
 'gs://ccle_bams/rna/G29750/HuNS1/v1/HuNS1.bam',
 'gs://ccle_bams/wes/G91718/KHYG-1_1/current/KHYG-1_1.bam',
 'gs://ccle_bams/rna/G16640/SNU-387/current/SNU-387.bam',
 'gs://ccle_bams/rna/G29750/FTC-133/v2/FTC-133.bam',
 'gs://ccle_bams/rna/G16640/SNU-1272/current/SNU-1272.bam',
 'gs://ccle_bams/rna/G16640/GA-10/current/GA-10.bam',
 'gs://ccle_bams/wes/G41775/A1207/current/A1207.bam',
 'gs://ccle_bams/rna/G16640/JHOM-2B/current/JHOM-2B.bam',
 'gs://ccle_bams/wes/G41786/TIG3-TD/current/TIG3-TD.bam',
 'gs://ccle_bams/rna/G48942/TE-14/current/TE-14.bam',
 'gs://ccle_bams/rna/G16640/NALM-19/current/NALM-19.bam',
 'gs://ccle_bams/rna/G29750/Hs_294T/v3/Hs_294T.bam',
 'gs://ccle_bams/rna/G29750/TEN/v1/TEN.bam',
 'gs://ccle_bams/rna/G29750/U-2_OS/v1/U-2_OS.bam',
 'gs://ccle_bams/wes/G146853/SKGI/v2/SKGI.bam',
 'gs://ccle_bams/rna/G29750/SW_900/v1/SW_900.bam',
 'gs://ccle_bams/rna/C1926/Hs_675.T/current/Hs_675.T.bam',
 'gs://ccle_bams/rna/G16640/CCK-81/current/CCK-81.bam',
 'gs://ccle_bams/rna/G29750/SNU-5/v1/SNU-5.bam',
 'gs://ccle_bams/rna/G29750/OVKATE/v3/OVKATE.bam',
 'gs://ccle_bams/rna/C1926/SNU-620/current/SNU-620.bam',
 'gs://ccle_bams/rna/G16640/SNU-407/current/SNU-407.bam',
 'gs://ccle_bams/rna/G29750/SNU-503/v3/SNU-503.bam',
 'gs://ccle_bams/rna/G16640/SW_1271/current/SW_1271.bam',
 'gs://ccle_bams/wes/G28813.Hs_274.T.bam',
 'gs://ccle_bams/rna/G29750/SNU-449/v1/SNU-449.bam',
 'gs://ccle_bams/rna/G29750/A-375/v1/A-375.bam',
 'gs://ccle_bams/rna/G29750/OV56/v2/OV56.bam',
 'gs://ccle_bams/rna/G32831/UACC-893/v1/UACC-893.bam',
 'gs://ccle_bams/wes/G131709/SiHa/current/SiHa.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1703/current/NCI-H1703.bam',
 'gs://ccle_bams/rna/G16465/G-361/v1/G-361.bam',
 'gs://ccle_bams/rna/G16640/JMSU-1/current/JMSU-1.bam',
 'gs://ccle_bams/rna/G29750/DM-3/v1/DM-3.bam',
 'gs://ccle_bams/rna/G16640/TE-6/current/TE-6.bam',
 'gs://ccle_bams/rna/G29750/LU99/v1/LU99.bam',
 'gs://ccle_bams/rna/G16370/KYSE-180/v2/KYSE-180.bam',
 'gs://ccle_bams/rna/G16640/L-540/current/L-540.bam',
 'gs://ccle_bams/wes/G93756/DL-40_1/current/DL-40_1.bam',
 'gs://ccle_bams/rna/G29750/MOLP-2/v2/MOLP-2.bam',
 'gs://ccle_bams/rna/G29750/OE33/v2/OE33.bam',
 'gs://ccle_bams/rna/G29750/MSTO-211H/v1/MSTO-211H.bam',
 'gs://ccle_bams/rna/G29750/CL-11/v1/CL-11.bam',
 'gs://ccle_bams/rna/G29750/ME-1/v3/ME-1.bam',
 'gs://ccle_bams/rna/G16640/Capan-1/current/Capan-1.bam',
 'gs://ccle_bams/rna/G16417/SNG-M/v1/SNG-M.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1651/v1/NCI-H1651.bam',
 'gs://ccle_bams/rna/G16640/Hs_688_A_.T/current/Hs_688_A_.T.bam',
 'gs://ccle_bams/rna/G29750/MIA_PaCa-2/v3/MIA_PaCa-2.bam',
 'gs://ccle_bams/rna/G32831/CAL-120/v2/CAL-120.bam',
 'gs://ccle_bams/rna/C1926/SNU-719/current/SNU-719.bam',
 'gs://ccle_bams/rna/G16640/GCT/current/GCT.bam',
 'gs://ccle_bams/rna/G16640/JHOS-4/current/JHOS-4.bam',
 'gs://ccle_bams/rna/G16640/EJM/current/EJM.bam',
 'gs://ccle_bams/rna/G32831/MDA-MB-134-VI/v1/MDA-MB-134-VI.bam',
 'gs://ccle_bams/rna/C1926/RERF-LC-KJ/current/RERF-LC-KJ.bam',
 'gs://ccle_bams/rna/G32831/COV362/v2/COV362.bam',
 'gs://ccle_bams/wes/G130264/127399_1/current/127399_1.bam',
 'gs://ccle_bams/rna/G32831/HuTu_80/v4/HuTu_80.bam',
 'gs://ccle_bams/rna/G16364/NCI-H747/v2/NCI-H747.bam',
 'gs://ccle_bams/rna/G29750/Sq-1/v3/Sq-1.bam',
 'gs://ccle_bams/rna/G29750/SNU-869/v1/SNU-869.bam',
 'gs://ccle_bams/rna/G29750/A4_Fuk/v2/A4_Fuk.bam',
 'gs://ccle_bams/rna/G16640/LUDLU-1/current/LUDLU-1.bam',
 'gs://ccle_bams/rna/G29750/HCC-33/v3/HCC-33.bam',
 'gs://ccle_bams/rna/G29750/TE-11/v1/TE-11.bam',
 'gs://ccle_bams/rna/G29750/HOS/v1/HOS.bam',
 'gs://ccle_bams/rna/G16640/Hs_888.T/current/Hs_888.T.bam',
 'gs://ccle_bams/rna/G16445/HSC-3/v1/HSC-3.bam',
 'gs://ccle_bams/rna/G32831/SK-MEL-24/v2/SK-MEL-24.bam',
 'gs://ccle_bams/rna/G29750/HCC2935/v3/HCC2935.bam',
 'gs://ccle_bams/rna/C1926/LUDLU-1/current/LUDLU-1.bam',
 'gs://ccle_bams/rna/G29750/NALM-1/v3/NALM-1.bam',
 'gs://ccle_bams/rna/G29750/FaDu/v1/FaDu.bam',
 'gs://ccle_bams/rna/G16640/JHOM-1/current/JHOM-1.bam',
 'gs://ccle_bams/rna/G16640/HCC1954/current/HCC1954.bam',
 'gs://ccle_bams/rna/G29750/MOLT-13/v1/MOLT-13.bam',
 'gs://ccle_bams/rna/G16640/NB-4/current/NB-4.bam',
 'gs://ccle_bams/rna/G29750/JIMT-1/v1/JIMT-1.bam',
 'gs://ccle_bams/rna/G29750/OS-RC-2/v1/OS-RC-2.bam',
 'gs://ccle_bams/rna/G16640/SNU-466/current/SNU-466.bam',
 'gs://ccle_bams/wes/G144963/KMLS1/current/KMLS1.bam',
 'gs://ccle_bams/rna/G29750/SNU-C2A/v3/SNU-C2A.bam',
 'gs://ccle_bams/rna/C1926/SNU-81/current/SNU-81.bam',
 'gs://ccle_bams/rna/G16640/KNS-60/current/KNS-60.bam',
 'gs://ccle_bams/rna/G29750/HEC-1-B/v2/HEC-1-B.bam',
 'gs://ccle_bams/rna/G29750/GP2d/v1/GP2d.bam',
 'gs://ccle_bams/rna/G16640/NCI-H510/current/NCI-H510.bam',
 'gs://ccle_bams/rna/G32831/BT-474/v1/BT-474.bam',
 'gs://ccle_bams/rna/G16640/8305C/current/8305C.bam',
 'gs://ccle_bams/rna/G16640/ML-1/current/ML-1.bam',
 'gs://ccle_bams/rna/G16640/VMRC-RCZ/current/VMRC-RCZ.bam',
 'gs://ccle_bams/rna/G16640/HPAC/current/HPAC.bam',
 'gs://ccle_bams/rna/G29750/F-36P/v1/F-36P.bam',
 'gs://ccle_bams/rna/C1926/SK-MEL-28/current/SK-MEL-28.bam',
 'gs://ccle_bams/rna/G16640/KYSE-510/current/KYSE-510.bam',
 'gs://ccle_bams/rna/G16640/NCI-H508/current/NCI-H508.bam',
 'gs://ccle_bams/rna/G29750/A549/v2/A549.bam',
 'gs://ccle_bams/rna/G16640/PE_CA-PJ49/current/PE_CA-PJ49.bam',
 'gs://ccle_bams/rna/G29750/SNU-601/v3/SNU-601.bam',
 'gs://ccle_bams/rna/G16640/LU99/current/LU99.bam',
 'gs://ccle_bams/rna/G29750/HuH28/v4/HuH28.bam',
 'gs://ccle_bams/wes/G106825/TC138_1/current/TC138_1.bam',
 'gs://ccle_bams/wes/G134624/MERO83_1/current/MERO83_1.bam',
 'gs://ccle_bams/rna/G16640/Malme-3M/current/Malme-3M.bam',
 'gs://ccle_bams/rna/G32831/COV644/v2/COV644.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1339/current/NCI-H1339.bam',
 'gs://ccle_bams/rna/G16640/HCC1937/current/HCC1937.bam',
 'gs://ccle_bams/rna/G16640/HuNS1/current/HuNS1.bam',
 'gs://ccle_bams/rna/G16640/KYSE-70/current/KYSE-70.bam',
 'gs://ccle_bams/rna/G29750/SNU-1079/v5/SNU-1079.bam',
 'gs://ccle_bams/wes/G91718/NKL_1/current/NKL_1.bam',
 'gs://ccle_bams/rna/C1926/NALM-19/current/NALM-19.bam',
 'gs://ccle_bams/rna/G29750/SIG-M5/v3/SIG-M5.bam',
 'gs://ccle_bams/rna/G29750/SW1116/v2/SW1116.bam',
 'gs://ccle_bams/wes/G41726.MCF7.5.bam',
 'gs://ccle_bams/rna/C1926/COLO-678/current/COLO-678.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2073/current/NCI-H2073.bam',
 'gs://ccle_bams/rna/G29750/SK-MEL-1/v2/SK-MEL-1.bam',
 'gs://ccle_bams/rna/G29750/SEM/v2/SEM.bam',
 'gs://ccle_bams/rna/G16640/MKN7/current/MKN7.bam',
 'gs://ccle_bams/rna/G32831/OE19/v2/OE19.bam',
 'gs://ccle_bams/wes/U343/current/U343.bam',
 'gs://ccle_bams/rna/C1926/HT55/current/HT55.bam',
 'gs://ccle_bams/rna/G16640/KNS-62/current/KNS-62.bam',
 'gs://ccle_bams/rna/G16640/MHH-CALL-3/current/MHH-CALL-3.bam',
 'gs://ccle_bams/rna/G29750/LN-229/v1/LN-229.bam',
 'gs://ccle_bams/rna/G48942/SCLC-21H/current/SCLC-21H.bam',
 'gs://ccle_bams/wes/G18893/TC32/current/TC32.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2030/current/NCI-H2030.bam',
 'gs://ccle_bams/rna/G29750/NCI-H358/v1/NCI-H358.bam',
 'gs://ccle_bams/rna/C1926/SNU-520/current/SNU-520.bam',
 'gs://ccle_bams/rna/G16640/SNU-201/current/SNU-201.bam',
 'gs://ccle_bams/rna/G16640/PF-382/current/PF-382.bam',
 'gs://ccle_bams/rna/G16442/EFO-27/v1/EFO-27.bam',
 'gs://ccle_bams/rna/G16640/HEC-1-B/current/HEC-1-B.bam',
 'gs://ccle_bams/rna/G32831/RPMI-7951/v2/RPMI-7951.bam',
 'gs://ccle_bams/rna/G32831/A2058/v1/A2058.bam',
 'gs://ccle_bams/rna/G29750/RL95-2/v2/RL95-2.bam',
 'gs://ccle_bams/rna/G16640/LCLC-97TM1/current/LCLC-97TM1.bam',
 'gs://ccle_bams/rna/G29750/MEG-01/v1/MEG-01.bam',
 'gs://ccle_bams/rna/C1926/SNU-1197/current/SNU-1197.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1155/current/NCI-H1155.bam',
 'gs://ccle_bams/rna/G16640/HT115/current/HT115.bam',
 'gs://ccle_bams/rna/G29750/TT/v4/TT.bam',
 'gs://ccle_bams/rna/G32831/HCC1419/v2/HCC1419.bam',
 'gs://ccle_bams/rna/G29750/TGBC11TKB/v1/TGBC11TKB.bam',
 'gs://ccle_bams/wes/G144159/NMB/v1/NMB.bam',
 'gs://ccle_bams/wes/G134624/ESO26_1/current/ESO26_1.bam',
 'gs://ccle_bams/rna/G29750/CL-40/v1/CL-40.bam',
 'gs://ccle_bams/rna/G16640/SCLC-21H/current/SCLC-21H.bam',
 'gs://ccle_bams/wes/G141708/C33A/current/C33A.bam',
 'gs://ccle_bams/wes/G106825/COGN305_1/current/COGN305_1.bam',
 'gs://ccle_bams/rna/G29750/KYSE-140/v1/KYSE-140.bam',
 'gs://ccle_bams/rna/G29750/CA46/v2/CA46.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1618/v1/NCI-H1618.bam',
 'gs://ccle_bams/rna/G32831/GP2d/v1/GP2d.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1373/current/NCI-H1373.bam',
 'gs://ccle_bams/rna/G16640/KMRC-3/current/KMRC-3.bam',
 'gs://ccle_bams/wes/G130264/COV504_1/current/COV504_1.bam',
 'gs://ccle_bams/wes/G136933/SCMCRM2_1/current/SCMCRM2_1.bam',
 'gs://ccle_bams/rna/G29750/SW_1783/v2/SW_1783.bam',
 'gs://ccle_bams/rna/G16640/SNU-1105/current/SNU-1105.bam',
 'gs://ccle_bams/rna/G16640/KE-39/current/KE-39.bam',
 'gs://ccle_bams/rna/G16640/KALS-1/current/KALS-1.bam',
 'gs://ccle_bams/wes/G77556/MON_01/current/MON_01.bam',
 'gs://ccle_bams/rna/G16640/BFTC-905/current/BFTC-905.bam',
 'gs://ccle_bams/rna/C817/TTC-466/current/TTC-466.bam',
 'gs://ccle_bams/wes/G27496.SNU-216.2.bam',
 'gs://ccle_bams/rna/G16640/NH-6/current/NH-6.bam',
 'gs://ccle_bams/rna/G16640/MV-4-11/current/MV-4-11.bam',
 'gs://ccle_bams/rna/G16640/DK-MG/current/DK-MG.bam',
 'gs://ccle_bams/rna/G16387/HCC-33/v1/HCC-33.bam',
 'gs://ccle_bams/rna/G29750/VM-CUB1/v2/VM-CUB1.bam',
 'gs://ccle_bams/rna/G16640/A549/current/A549.bam',
 'gs://ccle_bams/rna/G16640/J82/current/J82.bam',
 'gs://ccle_bams/rna/G32831/OVCAR-4/v2/OVCAR-4.bam',
 'gs://ccle_bams/rna/G16640/Hs_675.T/current/Hs_675.T.bam',
 'gs://ccle_bams/rna/G29750/MDA-MB-415/v2/MDA-MB-415.bam',
 'gs://ccle_bams/rna/G32831/769-P/v1/769-P.bam',
 'gs://ccle_bams/rna/G16640/KMRC-2/current/KMRC-2.bam',
 'gs://ccle_bams/rna/G48942/WM1799/current/WM1799.bam',
 'gs://ccle_bams/rna/G16640/KMRC-20/current/KMRC-20.bam',
 'gs://ccle_bams/wes/G134624/COLO794_1/current/COLO794_1.bam',
 'gs://ccle_bams/rna/G16640/G-402/current/G-402.bam',
 'gs://ccle_bams/wes/G134624/SCLC22H_1/current/SCLC22H_1.bam',
 'gs://ccle_bams/rna/G29750/SNU-407/v1/SNU-407.bam',
 'gs://ccle_bams/rna/G16640/OS-RC-2/current/OS-RC-2.bam',
 'gs://ccle_bams/rna/G29750/SW_1990/v1/SW_1990.bam',
 'gs://ccle_bams/rna/G16640/JHH-6/current/JHH-6.bam',
 'gs://ccle_bams/wes/G134624/C99_1/current/C99_1.bam',
 'gs://ccle_bams/rna/G16640/MFE-280/current/MFE-280.bam',
 'gs://ccle_bams/rna/G16640/PK-59/current/PK-59.bam',
 'gs://ccle_bams/rna/C1926/IST-MES2/current/IST-MES2.bam',
 'gs://ccle_bams/rna/G16640/LN-229/current/LN-229.bam',
 'gs://ccle_bams/rna/G32831/Calu-6/v2/Calu-6.bam',
 'gs://ccle_bams/rna/G29750/DMS_53/v2/DMS_53.bam',
 'gs://ccle_bams/rna/G29750/BFTC-905/v2/BFTC-905.bam',
 'gs://ccle_bams/rna/G29750/JURL-MK1/v2/JURL-MK1.bam',
 'gs://ccle_bams/rna/C1926/HSC-4/current/HSC-4.bam',
 'gs://ccle_bams/rna/G16640/BCP-1/current/BCP-1.bam',
 'gs://ccle_bams/rna/G16640/BDCM/current/BDCM.bam',
 'gs://ccle_bams/rna/G16405/NCI-H650/v1/NCI-H650.bam',
 'gs://ccle_bams/rna/G16640/P3HR-1/current/P3HR-1.bam',
 'gs://ccle_bams/rna/C1926/SK-N-MC/current/SK-N-MC.bam',
 'gs://ccle_bams/rna/G29750/DU4475/v1/DU4475.bam',
 'gs://ccle_bams/rna/G16640/LC-1F/current/LC-1F.bam',
 'gs://ccle_bams/rna/C1926/SNU-283/current/SNU-283.bam',
 'gs://ccle_bams/rna/G29750/U-BLC1/v2/U-BLC1.bam',
 'gs://ccle_bams/wes/G131709/SNU-410/current/SNU-410.bam',
 'gs://ccle_bams/rna/G32831/HEC-1-A/v1/HEC-1-A.bam',
 'gs://ccle_bams/rna/C836/K-562/v2/K-562.bam',
 'gs://ccle_bams/rna/G48942/NCI-H23/current/NCI-H23.bam',
 'gs://ccle_bams/rna/C1926/NCC-StC-K140/current/NCC-StC-K140.bam',
 'gs://ccle_bams/rna/G29750/639-V/v1/639-V.bam',
 'gs://ccle_bams/rna/G16640/KMM-1/current/KMM-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H146/v1/NCI-H146.bam',
 'gs://ccle_bams/rna/G29750/COLO_684/v1/COLO_684.bam',
 'gs://ccle_bams/rna/G29750/OUMS-27/v2/OUMS-27.bam',
 'gs://ccle_bams/wes/G134624/MM386_1/v4/MM386_1.bam',
 'gs://ccle_bams/rna/G29750/AU565/v2/AU565.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1568/current/NCI-H1568.bam',
 'gs://ccle_bams/rna/G16640/BHT-101/current/BHT-101.bam',
 'gs://ccle_bams/rna/G29750/SK-MEL-30/v1/SK-MEL-30.bam',
 'gs://ccle_bams/rna/G16640/NCI-H211/current/NCI-H211.bam',
 'gs://ccle_bams/rna/G16640/OV-90/current/OV-90.bam',
 'gs://ccle_bams/rna/G16640/SNU-899/current/SNU-899.bam',
 'gs://ccle_bams/wes/G141708/PACADD165/v2/PACADD165.bam',
 'gs://ccle_bams/rna/C1926/SNU-C1/current/SNU-C1.bam',
 'gs://ccle_bams/rna/G32831/Ishikawa__Heraklio__02_ER-/v1/Ishikawa__Heraklio__02_ER-.bam',
 'gs://ccle_bams/rna/G29750/KYSE-180/v1/KYSE-180.bam',
 'gs://ccle_bams/rna/G16640/COR-L47/current/COR-L47.bam',
 'gs://ccle_bams/wes/G102426/PEDS005PTFAD_1/current/PEDS005PTFAD_1.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2085/current/NCI-H2085.bam',
 'gs://ccle_bams/rna/G29750/DMS_114/v1/DMS_114.bam',
 'gs://ccle_bams/rna/G32831/Detroit_562/v2/Detroit_562.bam',
 'gs://ccle_bams/rna/G16640/Hs_936.T/current/Hs_936.T.bam',
 'gs://ccle_bams/rna/G29750/Hs_746T/v2/Hs_746T.bam',
 'gs://ccle_bams/rna/G16383/NCI-H510/v1/NCI-H510.bam',
 'gs://ccle_bams/rna/G16640/Set-2/current/Set-2.bam',
 'gs://ccle_bams/rna/G29750/S-117/v2/S-117.bam',
 'gs://ccle_bams/rna/G16640/KMS-20/current/KMS-20.bam',
 'gs://ccle_bams/rna/G29750/G-292,_clone_A141B1/v1/G-292,_clone_A141B1.bam',
 'gs://ccle_bams/rna/G29750/DU_145/v2/DU_145.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1792/v1/NCI-H1792.bam',
 'gs://ccle_bams/rna/G29750/HSC-2/v1/HSC-2.bam',
 'gs://ccle_bams/rna/G29750/D341_Med/v2/D341_Med.bam',
 'gs://ccle_bams/rna/G29750/SK-LMS-1/v3/SK-LMS-1.bam',
 'gs://ccle_bams/rna/G16640/EBC-1/current/EBC-1.bam',
 'gs://ccle_bams/rna/G29750/HCC70/v1/HCC70.bam',
 'gs://ccle_bams/rna/C1926/HCC-1195/current/HCC-1195.bam',
 'gs://ccle_bams/rna/G32831/VM-CUB1/v2/VM-CUB1.bam',
 'gs://ccle_bams/rna/G16640/DOHH-2/current/DOHH-2.bam',
 'gs://ccle_bams/rna/G16640/OCI-M1/current/OCI-M1.bam',
 'gs://ccle_bams/rna/G16355/NCI-H2170/v1/NCI-H2170.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1385/current/NCI-H1385.bam',
 'gs://ccle_bams/rna/G29750/SH-10-TC/v3/SH-10-TC.bam',
 'gs://ccle_bams/rna/G29750/EFE-184/v2/EFE-184.bam',
 'gs://ccle_bams/rna/G16640/BICR_56/current/BICR_56.bam',
 'gs://ccle_bams/wes/G41789/EW8/current/EW8.bam',
 'gs://ccle_bams/rna/G32831/Caov-3/v2/Caov-3.bam',
 'gs://ccle_bams/rna/G48942/SCC-4/current/SCC-4.bam',
 'gs://ccle_bams/rna/G16640/SNU-520/current/SNU-520.bam',
 'gs://ccle_bams/rna/Getz_IBM_CellLines_RNASeqData_1sample_09262017/C1926/NA/SW_1573/v1/SW_1573.bam',
 'gs://ccle_bams/wes/G141708/PACADD188/current/PACADD188.bam',
 'gs://ccle_bams/wes/G134624/SUM149PT_1/current/SUM149PT_1.bam',
 'gs://ccle_bams/rna/G32831/SNU-407/v1/SNU-407.bam',
 'gs://ccle_bams/rna/G16640/C32/current/C32.bam',
 'gs://ccle_bams/wes/G144159/PACADD159/current/PACADD159.bam',
 'gs://ccle_bams/rna/G16640/T-47D/current/T-47D.bam',
 'gs://ccle_bams/rna/G16640/KMS-21BM/current/KMS-21BM.bam',
 'gs://ccle_bams/rna/G16640/SNU-840/current/SNU-840.bam',
 'gs://ccle_bams/rna/G29750/HCC-1359/v2/HCC-1359.bam',
 'gs://ccle_bams/rna/G29750/Mino/v1/Mino.bam',
 'gs://ccle_bams/rna/G32831/KU-19-19/v3/KU-19-19.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2405/v2/NCI-H2405.bam',
 'gs://ccle_bams/wes/G141708/MS751/v2/MS751.bam',
 'gs://ccle_bams/rna/G29750/HPAC/v2/HPAC.bam',
 'gs://ccle_bams/rna/G16640/Toledo/current/Toledo.bam',
 'gs://ccle_bams/rna/G29750/SW-1710/v1/SW-1710.bam',
 'gs://ccle_bams/rna/G16640/HCC38/current/HCC38.bam',
 'gs://ccle_bams/rna/G29750/TE-9/v1/TE-9.bam',
 'gs://ccle_bams/rna/G16640/AN3_CA/current/AN3_CA.bam',
 'gs://ccle_bams/wes/G141708/PA1/current/PA1.bam',
 'gs://ccle_bams/rna/G29750/HuH-6/v4/HuH-6.bam',
 'gs://ccle_bams/rna/G29750/59M/v3/59M.bam',
 'gs://ccle_bams/rna/G48942/HCC1599/current/HCC1599.bam',
 'gs://ccle_bams/rna/G16640/TUHR4TKB/current/TUHR4TKB.bam',
 'gs://ccle_bams/rna/G16381/DMS_153/v1/DMS_153.bam',
 'gs://ccle_bams/rna/G16640/EC-GI-10/current/EC-GI-10.bam',
 'gs://ccle_bams/rna/G32831/DU4475/v1/DU4475.bam',
 'gs://ccle_bams/rna/G16640/SNU-719/current/SNU-719.bam',
 'gs://ccle_bams/rna/G16409/NCI-H2122/v1/NCI-H2122.bam',
 'gs://ccle_bams/rna/G16640/LOX_IMVI/current/LOX_IMVI.bam',
 'gs://ccle_bams/rna/G32831/JHUEM-1/v2/JHUEM-1.bam',
 'gs://ccle_bams/wes/G25203.NCI-H1092.1.bam',
 'gs://ccle_bams/rna/G29750/MOLM-13/v2/MOLM-13.bam',
 'gs://ccle_bams/rna/G29750/UM-UC-1/v5/UM-UC-1.bam',
 'gs://ccle_bams/rna/C1926/Hs_600.T/current/Hs_600.T.bam',
 'gs://ccle_bams/rna/G29750/COLO_829/v2/COLO_829.bam',
 'gs://ccle_bams/rna/G29750/PE_CA-PJ34__clone_C12_/v2/PE_CA-PJ34__clone_C12_.bam',
 'gs://ccle_bams/rna/G16640/CAL-33/current/CAL-33.bam',
 'gs://ccle_bams/rna/G29750/BL-70/v2/BL-70.bam',
 'gs://ccle_bams/rna/C1926/CW-2/current/CW-2.bam',
 'gs://ccle_bams/rna/G29750/PF-382/v1/PF-382.bam',
 'gs://ccle_bams/rna/G29750/NH-6/v6/NH-6.bam',
 'gs://ccle_bams/wes/G134624/MCC13_1/current/MCC13_1.bam',
 'gs://ccle_bams/rna/C1926/COLO_792/current/COLO_792.bam',
 'gs://ccle_bams/wes/G20474.Hs_746T.2.bam',
 'gs://ccle_bams/rna/G29750/ML-1/v2/ML-1.bam',
 'gs://ccle_bams/rna/G16640/MCAS/current/MCAS.bam',
 'gs://ccle_bams/rna/G29750/LAMA-84/v2/LAMA-84.bam',
 'gs://ccle_bams/rna/G32831/SCaBER/v2/SCaBER.bam',
 'gs://ccle_bams/rna/G32831/LoVo/v1/LoVo.bam',
 'gs://ccle_bams/rna/G16640/NCI-H146/current/NCI-H146.bam',
 'gs://ccle_bams/rna/G16640/KYO-1/current/KYO-1.bam',
 'gs://ccle_bams/wes/G134624/SUM299PE_1/current/SUM299PE_1.bam',
 'gs://ccle_bams/wes/G41801.Caco-2.bam',
 'gs://ccle_bams/rna/G29750/JHUEM-1/v2/JHUEM-1.bam',
 'gs://ccle_bams/rna/G29750/OCI-AML3/v2/OCI-AML3.bam',
 'gs://ccle_bams/rna/G29750/HCC-78/v2/HCC-78.bam',
 'gs://ccle_bams/rna/G29750/YD-15/v1/YD-15.bam',
 'gs://ccle_bams/rna/G16481/DU_145/v1/DU_145.bam',
 'gs://ccle_bams/wes/G131709/LN-428/current/LN-428.bam',
 'gs://ccle_bams/wes/G41770.SLR25.bam',
 'gs://ccle_bams/rna/G16640/RL95-2/current/RL95-2.bam',
 'gs://ccle_bams/rna/G29750/HCC-1833/v3/HCC-1833.bam',
 'gs://ccle_bams/rna/G16459/OPM-2/v2/OPM-2.bam',
 'gs://ccle_bams/rna/G29750/MKN1/v1/MKN1.bam',
 'gs://ccle_bams/rna/C1926/GSU/current/GSU.bam',
 'gs://ccle_bams/rna/G16640/KARPAS-620/current/KARPAS-620.bam',
 'gs://ccle_bams/rna/G32831/EFM-192A/v1/EFM-192A.bam',
 'gs://ccle_bams/rna/G16640/SNU-869/current/SNU-869.bam',
 'gs://ccle_bams/wes/G35070/OVKATE/current/OVKATE.bam',
 'gs://ccle_bams/wes/G102426/D458_1/current/D458_1.bam',
 'gs://ccle_bams/wes/G35076/NCI-H1373/current/NCI-H1373.bam',
 'gs://ccle_bams/rna/C817/EWS-502/current/EWS-502.bam',
 'gs://ccle_bams/wes/G143832/OC316/current/OC316.bam',
 'gs://ccle_bams/wes/G28857.HuH28.bam',
 'gs://ccle_bams/rna/C1926/MDST8/current/MDST8.bam',
 'gs://ccle_bams/wes/G134624/MERO48A_1/v4/MERO48A_1.bam',
 'gs://ccle_bams/rna/G29750/697/v2/697.bam',
 'gs://ccle_bams/rna/G16640/ZR-75-1/current/ZR-75-1.bam',
 'gs://ccle_bams/rna/G29750/NMC-G1/v1/NMC-G1.bam',
 'gs://ccle_bams/rna/G29750/RERF-LC-KJ/v2/RERF-LC-KJ.bam',
 'gs://ccle_bams/wes/G106825/CHLA266_1/current/CHLA266_1.bam',
 'gs://ccle_bams/rna/C1926/A-673/current/A-673.bam',
 'gs://ccle_bams/wes/G106825/CHLA57_1/current/CHLA57_1.bam',
 'gs://ccle_bams/rna/G29750/BICR_31/v3/BICR_31.bam',
 'gs://ccle_bams/rna/G29750/Hs_578T/v2/Hs_578T.bam',
 'gs://ccle_bams/rna/G32831/MONO-MAC-1/v2/MONO-MAC-1.bam',
 'gs://ccle_bams/rna/G16640/HCC-1833/current/HCC-1833.bam',
 'gs://ccle_bams/wes/G144963/SW954/v2/SW954.bam',
 'gs://ccle_bams/rna/G29750/DEL/v3/DEL.bam',
 'gs://ccle_bams/rna/G29750/AMO-1/v2/AMO-1.bam',
 'gs://ccle_bams/wes/G134624/MM426_1/v4/MM426_1.bam',
 'gs://ccle_bams/rna/G16640/OAW28/current/OAW28.bam',
 'gs://ccle_bams/rna/G32831/GI-1/v1/GI-1.bam',
 'gs://ccle_bams/wes/G41759/SF767/current/SF767.bam',
 'gs://ccle_bams/rna/G16640/SNU-1077/current/SNU-1077.bam',
 'gs://ccle_bams/rna/G29750/LUDLU-1/v1/LUDLU-1.bam',
 'gs://ccle_bams/rna/G32831/ChaGo-K-1/v2/ChaGo-K-1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1876/current/NCI-H1876.bam',
 'gs://ccle_bams/rna/C1926/NCI-H441/current/NCI-H441.bam',
 'gs://ccle_bams/rna/G16640/DOV13/current/DOV13.bam',
 'gs://ccle_bams/rna/G16640/Hs_604.T/current/Hs_604.T.bam',
 'gs://ccle_bams/rna/G29750/RH-30/v2/RH-30.bam',
 'gs://ccle_bams/rna/G16427/Hep_G2/v1/Hep_G2.bam',
 'gs://ccle_bams/rna/G29750/Raji/v2/Raji.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2347/current/NCI-H2347.bam',
 'gs://ccle_bams/rna/G29750/VMRC-RCW/v1/VMRC-RCW.bam',
 'gs://ccle_bams/rna/G16640/SNU-398/current/SNU-398.bam',
 'gs://ccle_bams/rna/G29750/GSS/v1/GSS.bam',
 'gs://ccle_bams/rna/G16360/TE-11/v1/TE-11.bam',
 'gs://ccle_bams/rna/G29750/Hs_936.T/v2/Hs_936.T.bam',
 'gs://ccle_bams/rna/G32831/Hs_294T/v1/Hs_294T.bam',
 'gs://ccle_bams/rna/G29750/L3.3/v2/L3.3.bam',
 'gs://ccle_bams/wes/G93756/MTA_1/current/MTA_1.bam',
 'gs://ccle_bams/rna/G16640/SNU-1033/current/SNU-1033.bam',
 'gs://ccle_bams/wes/G27270.TE-11.1.bam',
 'gs://ccle_bams/rna/G29750/WSU-DLCL2/v3/WSU-DLCL2.bam',
 'gs://ccle_bams/rna/G29750/NUGC-4/v2/NUGC-4.bam',
 'gs://ccle_bams/rna/G29750/SK-BR-3/v1/SK-BR-3.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2172/v4/NCI-H2172.bam',
 'gs://ccle_bams/rna/G29750/DK-MG/v2/DK-MG.bam',
 'gs://ccle_bams/rna/G16640/SUP-T1/current/SUP-T1.bam',
 'gs://ccle_bams/rna/G29750/KALS-1/v2/KALS-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1793/v1/NCI-H1793.bam',
 'gs://ccle_bams/rna/G29750/HT55/v2/HT55.bam',
 'gs://ccle_bams/rna/G29750/A2780/v2/A2780.bam',
 'gs://ccle_bams/wes/G91718/Karpas-384_1/current/Karpas-384_1.bam',
 'gs://ccle_bams/rna/G29750/SNU-1077/v2/SNU-1077.bam',
 'gs://ccle_bams/rna/G29750/PCM6/v3/PCM6.bam',
 'gs://ccle_bams/rna/G32831/MDA-MB-453/v1/MDA-MB-453.bam',
 'gs://ccle_bams/rna/G16640/A-253/current/A-253.bam',
 'gs://ccle_bams/wes/G141708/UMUC7/v2/UMUC7.bam',
 'gs://ccle_bams/rna/G16398/SW480/v2/SW480.bam',
 'gs://ccle_bams/rna/G29750/CAL-51/v1/CAL-51.bam',
 'gs://ccle_bams/rna/G29750/Toledo/v2/Toledo.bam',
 'gs://ccle_bams/rna/G29750/HDLM-2/v1/HDLM-2.bam',
 'gs://ccle_bams/rna/G16640/KU-19-19/current/KU-19-19.bam',
 'gs://ccle_bams/rna/C1926/SNU-46/current/SNU-46.bam',
 'gs://ccle_bams/rna/C1926/NCI-H322/current/NCI-H322.bam',
 'gs://ccle_bams/rna/C1926/SK-MEL-3/current/SK-MEL-3.bam',
 'gs://ccle_bams/rna/G16640/NCI-H28/current/NCI-H28.bam',
 'gs://ccle_bams/wes/G41804.NCI-H2004_RT_2.bam',
 'gs://ccle_bams/rna/G16640/HCC-15/current/HCC-15.bam',
 'gs://ccle_bams/rna/G16640/HT-144/current/HT-144.bam',
 'gs://ccle_bams/rna/G29750/SCaBER/v1/SCaBER.bam',
 'gs://ccle_bams/rna/C1926/T3M-10/current/T3M-10.bam',
 'gs://ccle_bams/wes/G134624/MERO84_1/current/MERO84_1.bam',
 'gs://ccle_bams/rna/G29750/SNU-1/v1/SNU-1.bam',
 'gs://ccle_bams/rna/G16467/AGS/v2/AGS.bam',
 'gs://ccle_bams/rna/G29750/NCI-H650/v2/NCI-H650.bam',
 'gs://ccle_bams/rna/G29750/22Rv1/v3/22Rv1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2291/current/NCI-H2291.bam',
 'gs://ccle_bams/rna/C1926/NB-1/current/NB-1.bam',
 'gs://ccle_bams/rna/G16408/NCI-H1963/v1/NCI-H1963.bam',
 'gs://ccle_bams/rna/G16640/KMS-11/current/KMS-11.bam',
 'gs://ccle_bams/rna/G29750/SW_1271/v3/SW_1271.bam',
 'gs://ccle_bams/rna/G29750/HuG1-N/v1/HuG1-N.bam',
 'gs://ccle_bams/wes/G102426/D425_1/current/D425_1.bam',
 'gs://ccle_bams/rna/G48942/UACC-893/current/UACC-893.bam',
 'gs://ccle_bams/rna/G29750/JVM-3/v1/JVM-3.bam',
 'gs://ccle_bams/rna/G16400/SNU-387/v1/SNU-387.bam',
 'gs://ccle_bams/wes/G41820/TM-87/current/TM-87.bam',
 'gs://ccle_bams/rna/G29750/A-498/v2/A-498.bam',
 'gs://ccle_bams/rna/G29750/KNS-62/v1/KNS-62.bam',
 'gs://ccle_bams/rna/C1926/LCLC-97TM1/current/LCLC-97TM1.bam',
 'gs://ccle_bams/rna/G29750/Daudi/v1/Daudi.bam',
 'gs://ccle_bams/rna/G48942/U251-MG/current/U251-MG.bam',
 'gs://ccle_bams/rna/G16640/HCT_116/current/HCT_116.bam',
 'gs://ccle_bams/rna/G29750/KYM-1/v1/KYM-1.bam',
 'gs://ccle_bams/rna/G29750/RCH-ACV/v1/RCH-ACV.bam',
 'gs://ccle_bams/rna/G16640/Hs_939.T/current/Hs_939.T.bam',
 'gs://ccle_bams/rna/G32831/EBC-1/v1/EBC-1.bam',
 'gs://ccle_bams/rna/G29750/SU.86.86/v2/SU.86.86.bam',
 'gs://ccle_bams/rna/G48942/RCC10RGB/current/RCC10RGB.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2081/current/NCI-H2081.bam',
 'gs://ccle_bams/rna/G29750/JJN-3/v3/JJN-3.bam',
 'gs://ccle_bams/rna/C1926/CL-11/current/CL-11.bam',
 'gs://ccle_bams/rna/C1926/TF-1/current/TF-1.bam',
 'gs://ccle_bams/rna/G29750/CAL-54/v1/CAL-54.bam',
 'gs://ccle_bams/rna/G16640/MEC-1/current/MEC-1.bam',
 'gs://ccle_bams/rna/G16640/CAL-51/current/CAL-51.bam',
 'gs://ccle_bams/rna/G16640/HT-1376/current/HT-1376.bam',
 'gs://ccle_bams/rna/G16428/NCI-H1092/v1/NCI-H1092.bam',
 'gs://ccle_bams/rna/C1926/MPP_89/current/MPP_89.bam',
 'gs://ccle_bams/rna/G29750/KYSE-450/v1/KYSE-450.bam',
 'gs://ccle_bams/rna/G29750/P12-ICHIKAWA/v1/P12-ICHIKAWA.bam',
 'gs://ccle_bams/rna/G29750/HT/v2/HT.bam',
 'gs://ccle_bams/rna/G32831/MDA-MB-361/v2/MDA-MB-361.bam',
 'gs://ccle_bams/rna/G29750/MFE-280/v1/MFE-280.bam',
 'gs://ccle_bams/rna/G29750/CCK-81/v1/CCK-81.bam',
 'gs://ccle_bams/wes/G131709/C8166/current/C8166.bam',
 'gs://ccle_bams/rna/G29750/IMR-32/v1/IMR-32.bam',
 'gs://ccle_bams/rna/G16640/SW403/current/SW403.bam',
 'gs://ccle_bams/rna/G16640/GSS/current/GSS.bam',
 'gs://ccle_bams/rna/G16640/SK-MEL-5/current/SK-MEL-5.bam',
 'gs://ccle_bams/rna/G16640/SNU-620/current/SNU-620.bam',
 'gs://ccle_bams/wes/G41686.KYSE-150.5.bam',
 'gs://ccle_bams/rna/G29750/G-402/v2/G-402.bam',
 'gs://ccle_bams/rna/G29750/BEN/v1/BEN.bam',
 'gs://ccle_bams/rna/G16640/Capan-2/current/Capan-2.bam',
 'gs://ccle_bams/rna/G29750/LMSU/v2/LMSU.bam',
 'gs://ccle_bams/rna/G16441/DB/v1/DB.bam',
 'gs://ccle_bams/rna/G16640/BxPC-3/current/BxPC-3.bam',
 'gs://ccle_bams/wes/G106825/COGN278_1/current/COGN278_1.bam',
 'gs://ccle_bams/wes/G26236.NCI-H716.2.bam',
 'gs://ccle_bams/wes/G25205.NCI-H2286.1.bam',
 'gs://ccle_bams/rna/G29750/SNU-245/v2/SNU-245.bam',
 'gs://ccle_bams/rna/G29750/SU-DHL-6/v2/SU-DHL-6.bam',
 'gs://ccle_bams/rna/G16480/NCI-H2052/v1/NCI-H2052.bam',
 'gs://ccle_bams/rna/G32831/REC-1/v2/REC-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1876/v2/NCI-H1876.bam',
 'gs://ccle_bams/rna/G29750/RD/v2/RD.bam',
 'gs://ccle_bams/rna/G29750/COV318/v2/COV318.bam',
 'gs://ccle_bams/rna/G16640/MM1-S/current/MM1-S.bam',
 'gs://ccle_bams/rna/G29750/Hs_675.T/v3/Hs_675.T.bam',
 'gs://ccle_bams/rna/C1926/HCC4006/current/HCC4006.bam',
 'gs://ccle_bams/rna/G16640/JVM-2/current/JVM-2.bam',
 'gs://ccle_bams/rna/G29750/253J/v2/253J.bam',
 'gs://ccle_bams/rna/G16640/DU_145/current/DU_145.bam',
 'gs://ccle_bams/rna/G29750/COLO_792/v1/COLO_792.bam',
 'gs://ccle_bams/rna/G29750/SU-DHL-5/v2/SU-DHL-5.bam',
 'gs://ccle_bams/rna/G29750/COLO-679/v1/COLO-679.bam',
 'gs://ccle_bams/rna/G32831/QGP-1/v2/QGP-1.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2110/current/NCI-H2110.bam',
 'gs://ccle_bams/rna/G29750/OCUM-1/v1/OCUM-1.bam',
 'gs://ccle_bams/rna/G16640/CAL_27/current/CAL_27.bam',
 'gs://ccle_bams/rna/G16640/MDA-MB-175-VII/current/MDA-MB-175-VII.bam',
 'gs://ccle_bams/rna/G29750/EM-2/v4/EM-2.bam',
 'gs://ccle_bams/rna/G29750/8305C/v1/8305C.bam',
 'gs://ccle_bams/wes/G130264/143B_1/current/143B_1.bam',
 'gs://ccle_bams/rna/C1926/Hs_695T/current/Hs_695T.bam',
 'gs://ccle_bams/rna/G16640/SIMA/current/SIMA.bam',
 'gs://ccle_bams/rna/G16640/CW-2/current/CW-2.bam',
 'gs://ccle_bams/rna/G16640/JHH-5/current/JHH-5.bam',
 'gs://ccle_bams/rna/G29750/JHUEM-7/v2/JHUEM-7.bam',
 'gs://ccle_bams/rna/G48942/LS411N/current/LS411N.bam',
 'gs://ccle_bams/wes/G141708/UMUC16/current/UMUC16.bam',
 'gs://ccle_bams/rna/G16640/786-O/current/786-O.bam',
 'gs://ccle_bams/rna/G16640/Hs_944.T/current/Hs_944.T.bam',
 'gs://ccle_bams/rna/C1926/Hs_706.T/current/Hs_706.T.bam',
 'gs://ccle_bams/rna/G32831/CAL-54/v1/CAL-54.bam',
 'gs://ccle_bams/rna/G32831/SK-MEL-2/v2/SK-MEL-2.bam',
 'gs://ccle_bams/rna/G29750/SR-786/v1/SR-786.bam',
 'gs://ccle_bams/wes/G141708/PACADD119/current/PACADD119.bam',
 'gs://ccle_bams/rna/G16640/JHOS-2/current/JHOS-2.bam',
 'gs://ccle_bams/wes/G26187/MONO-MAC-1/v3/MONO-MAC-1.bam',
 'gs://ccle_bams/rna/G16640/HEC-59/current/HEC-59.bam',
 'gs://ccle_bams/rna/G29750/KE-97/v1/KE-97.bam',
 'gs://ccle_bams/rna/G16339/U-87_MG/v1/U-87_MG.bam',
 'gs://ccle_bams/rna/G29750/TE-8/v2/TE-8.bam',
 'gs://ccle_bams/rna/G29750/KMM-1/v1/KMM-1.bam',
 'gs://ccle_bams/rna/G16640/MDA_PCa_2b/current/MDA_PCa_2b.bam',
 'gs://ccle_bams/rna/G16640/PE_CA-PJ41__clone_D2_/current/PE_CA-PJ41__clone_D2_.bam',
 'gs://ccle_bams/rna/G16640/A101D/current/A101D.bam',
 'gs://ccle_bams/rna/G29750/RERF-LC-Ad2/v1/RERF-LC-Ad2.bam',
 'gs://ccle_bams/rna/G16640/BL-70/current/BL-70.bam',
 'gs://ccle_bams/rna/C1926/KOPN-8/current/KOPN-8.bam',
 'gs://ccle_bams/rna/G29750/SNU-1105/v1/SNU-1105.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1581/v1/NCI-H1581.bam',
 'gs://ccle_bams/wes/G41784/HMEL/current/HMEL.bam',
 'gs://ccle_bams/rna/G29750/T24/v2/T24.bam',
 'gs://ccle_bams/rna/G32831/CAS-1/v1/CAS-1.bam',
 'gs://ccle_bams/rna/G29750/SBC-5/v2/SBC-5.bam',
 'gs://ccle_bams/rna/G32831/LS_180/v1/LS_180.bam',
 'gs://ccle_bams/rna/G29750/Hs_821.T/v2/Hs_821.T.bam',
 'gs://ccle_bams/rna/G29750/M-07e/v3/M-07e.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1184/v3/NCI-H1184.bam',
 'gs://ccle_bams/rna/G16640/SK-MEL-31/current/SK-MEL-31.bam',
 'gs://ccle_bams/rna/G16640/CHP-126/current/CHP-126.bam',
 'gs://ccle_bams/rna/G32831/SK-LU-1/v3/SK-LU-1.bam',
 'gs://ccle_bams/rna/G16422/NCI-H2286/v1/NCI-H2286.bam',
 'gs://ccle_bams/wes/G136933/RD_P3F__14_1/v2/RD_P3F__14_1.bam',
 'gs://ccle_bams/rna/G29750/MOLP-8/v2/MOLP-8.bam',
 'gs://ccle_bams/rna/G29750/HCC1569/v1/HCC1569.bam',
 'gs://ccle_bams/wes/G41809/NCI-H2077/current/NCI-H2077.bam',
 'gs://ccle_bams/wes/G28844.HEL.3.bam',
 'gs://ccle_bams/rna/G32831/KARPAS-299/v2/KARPAS-299.bam',
 'gs://ccle_bams/rna/G29750/LXF-289/v2/LXF-289.bam',
 'gs://ccle_bams/wes/G102426/R262_1/current/R262_1.bam',
 'gs://ccle_bams/rna/G16640/CAL-62/current/CAL-62.bam',
 'gs://ccle_bams/rna/G29750/Calu-6/v1/Calu-6.bam',
 'gs://ccle_bams/rna/G29750/KMS-12-BM/v1/KMS-12-BM.bam',
 'gs://ccle_bams/rna/G16640/MHH-ES-1/current/MHH-ES-1.bam',
 'gs://ccle_bams/rna/G16418/NCI-H1437/v1/NCI-H1437.bam',
 'gs://ccle_bams/rna/G16640/HCC-366/current/HCC-366.bam',
 'gs://ccle_bams/rna/G29750/DMS_273/v1/DMS_273.bam',
 'gs://ccle_bams/wes/G141708/GIMEN/current/GIMEN.bam',
 'gs://ccle_bams/wes/G144159/ONDA9/v1/ONDA9.bam',
 'gs://ccle_bams/rna/G48942/HCC1428/current/HCC1428.bam',
 'gs://ccle_bams/rna/G29750/YAPC/v1/YAPC.bam',
 'gs://ccle_bams/rna/G29750/BICR_16/v3/BICR_16.bam',
 'gs://ccle_bams/rna/G29750/SNU-1196/v2/SNU-1196.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2087/current/NCI-H2087.bam',
 'gs://ccle_bams/rna/G32831/OVK18/v2/OVK18.bam',
 'gs://ccle_bams/rna/G32831/LS411N/v1/LS411N.bam',
 'gs://ccle_bams/rna/C1926/Hs_698.T/current/Hs_698.T.bam',
 'gs://ccle_bams/rna/G16457/NUGC-4/v1/NUGC-4.bam',
 'gs://ccle_bams/rna/G29750/HCC2218/v1/HCC2218.bam',
 'gs://ccle_bams/rna/G29750/SNU-719/v4/SNU-719.bam',
 'gs://ccle_bams/rna/G16477/RERF-GC-1B/v1/RERF-GC-1B.bam',
 'gs://ccle_bams/rna/G16640/U-937/current/U-937.bam',
 'gs://ccle_bams/rna/G29750/HEC-50B/v2/HEC-50B.bam',
 'gs://ccle_bams/rna/G16374/HCT_116/v1/HCT_116.bam',
 'gs://ccle_bams/rna/G16640/COLO_684/current/COLO_684.bam',
 'gs://ccle_bams/wes/G144159/KKU100/v1/KKU100.bam',
 'gs://ccle_bams/rna/G16640/MDA-MB-435S/current/MDA-MB-435S.bam',
 'gs://ccle_bams/rna/G32831/EOL-1/v1/EOL-1.bam',
 'gs://ccle_bams/rna/G16640/RKO/current/RKO.bam',
 'gs://ccle_bams/rna/G16640/SNU-5/current/SNU-5.bam',
 'gs://ccle_bams/rna/G32831/KYSE-520/v1/KYSE-520.bam',
 'gs://ccle_bams/rna/G29750/RVH-421/v2/RVH-421.bam',
 'gs://ccle_bams/rna/C1926/MCAS/current/MCAS.bam',
 'gs://ccle_bams/wes/G77557.KD_01.bam',
 'gs://ccle_bams/wes/G134624/HSC1_1/current/HSC1_1.bam',
 'gs://ccle_bams/rna/G29750/RI-1/v2/RI-1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2085/current/NCI-H2085.bam',
 'gs://ccle_bams/rna/G29750/ALL-SIL/v3/ALL-SIL.bam',
 'gs://ccle_bams/rna/G29750/KU-19-19/v1/KU-19-19.bam',
 'gs://ccle_bams/rna/G16640/SNU-283/current/SNU-283.bam',
 'gs://ccle_bams/rna/G16436/U-2_OS/v1/U-2_OS.bam',
 'gs://ccle_bams/rna/G16640/HCC1806/current/HCC1806.bam',
 'gs://ccle_bams/wes/G130264/NB1643_1/current/NB1643_1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2023/current/NCI-H2023.bam',
 'gs://ccle_bams/rna/G16640/DBTRG-05MG/current/DBTRG-05MG.bam',
 'gs://ccle_bams/rna/G16640/SNU-601/current/SNU-601.bam',
 'gs://ccle_bams/rna/G29750/RT-112/v2/RT-112.bam',
 'gs://ccle_bams/wes/G134624/SEKI_1/current/SEKI_1.bam',
 'gs://ccle_bams/rna/G32831/UACC-812/v1/UACC-812.bam',
 'gs://ccle_bams/rna/G16640/RERF-LC-Sq1/current/RERF-LC-Sq1.bam',
 'gs://ccle_bams/wes/G27506.RERF-LC-KJ.2.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2227/v1/NCI-H2227.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1436/v1/NCI-H1436.bam',
 'gs://ccle_bams/rna/G29750/NCI-H661/v1/NCI-H661.bam',
 'gs://ccle_bams/rna/G16640/HCC2935/current/HCC2935.bam',
 'gs://ccle_bams/wes/G41776/JH-ESOAD1/current/JH-ESOAD1.bam',
 'gs://ccle_bams/rna/G16640/A-375/current/A-375.bam',
 'gs://ccle_bams/rna/G16640/SW1417/current/SW1417.bam',
 'gs://ccle_bams/rna/G16640/HUP-T3/current/HUP-T3.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2405/current/NCI-H2405.bam',
 'gs://ccle_bams/wes/G41778.SLR20.bam',
 'gs://ccle_bams/rna/G16640/CAL-12T/current/CAL-12T.bam',
 'gs://ccle_bams/rna/G32831/MDA-MB-175-VII/v2/MDA-MB-175-VII.bam',
 'gs://ccle_bams/rna/G29750/RMUG-S/v3/RMUG-S.bam',
 'gs://ccle_bams/rna/G29750/Malme-3M/v5/Malme-3M.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1048/current/NCI-H1048.bam',
 'gs://ccle_bams/rna/G32831/NCI-H23/v2/NCI-H23.bam',
 'gs://ccle_bams/rna/G16640/MUTZ-5/current/MUTZ-5.bam',
 'gs://ccle_bams/rna/G16375/NCI-H2029/v1/NCI-H2029.bam',
 'gs://ccle_bams/rna/C1926/ECC12/current/ECC12.bam',
 'gs://ccle_bams/rna/G32831/KYSE-70/v2/KYSE-70.bam',
 'gs://ccle_bams/rna/G16640/NIH_OVCAR-3/current/NIH_OVCAR-3.bam',
 'gs://ccle_bams/rna/G29750/Calu-1/v3/Calu-1.bam',
 'gs://ccle_bams/rna/G29750/HCC1143/v2/HCC1143.bam',
 'gs://ccle_bams/wes/G141708/UMUC11/current/UMUC11.bam',
 'gs://ccle_bams/rna/G29750/U266B1/v1/U266B1.bam',
 'gs://ccle_bams/rna/G16640/HPB-ALL/current/HPB-ALL.bam',
 'gs://ccle_bams/rna/G32831/NCI-H1355/v1/NCI-H1355.bam',
 'gs://ccle_bams/rna/G29750/SNU-738/v2/SNU-738.bam',
 'gs://ccle_bams/wes/G41796.HNT-34-2.bam',
 'gs://ccle_bams/rna/G16640/PCM6/current/PCM6.bam',
 'gs://ccle_bams/rna/G16640/SW1116/current/SW1116.bam',
 'gs://ccle_bams/rna/G29750/KP-2/v1/KP-2.bam',
 'gs://ccle_bams/rna/G29750/PLC_PRF_5/v2/PLC_PRF_5.bam',
 'gs://ccle_bams/rna/G16449/CHP-212/v2/CHP-212.bam',
 'gs://ccle_bams/rna/G16640/HEC-1-A/current/HEC-1-A.bam',
 'gs://ccle_bams/rna/G29750/Saos-2/v1/Saos-2.bam',
 'gs://ccle_bams/rna/G29750/VCaP/v2/VCaP.bam',
 'gs://ccle_bams/rna/G29750/KE-37/v1/KE-37.bam',
 'gs://ccle_bams/rna/G29750/KYSE-270/v1/KYSE-270.bam',
 'gs://ccle_bams/rna/G29750/OCI-LY-19/v2/OCI-LY-19.bam',
 'gs://ccle_bams/rna/G16640/SW837/current/SW837.bam',
 'gs://ccle_bams/rna/G29750/COR-L95/v2/COR-L95.bam',
 'gs://ccle_bams/rna/G16640/Calu-1/current/Calu-1.bam',
 'gs://ccle_bams/rna/G29750/BICR_56/v2/BICR_56.bam',
 'gs://ccle_bams/rna/G32831/HCC-95/v1/HCC-95.bam',
 'gs://ccle_bams/rna/G29750/AGS/v1/AGS.bam',
 'gs://ccle_bams/wes/G130264/PEDS092T__1/current/PEDS092T__1.bam',
 'gs://ccle_bams/rna/G29750/SU-DHL-10/v4/SU-DHL-10.bam',
 'gs://ccle_bams/rna/G16640/SU-DHL-8/current/SU-DHL-8.bam',
 'gs://ccle_bams/wes/G141708/UMUC6/v2/UMUC6.bam',
 'gs://ccle_bams/rna/C1926/TGBC11TKB/current/TGBC11TKB.bam',
 'gs://ccle_bams/rna/G16640/OCI-AML3/current/OCI-AML3.bam',
 'gs://ccle_bams/rna/G29750/T3M-10/v1/T3M-10.bam',
 'gs://ccle_bams/rna/G16640/JeKo-1/current/JeKo-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H441/v1/NCI-H441.bam',
 'gs://ccle_bams/rna/G16640/T.T/current/T.T.bam',
 'gs://ccle_bams/rna/G32831/A101D/v2/A101D.bam',
 'gs://ccle_bams/wes/G41765/SLR24/current/SLR24.bam',
 'gs://ccle_bams/rna/G29750/HPAF-II/v1/HPAF-II.bam',
 'gs://ccle_bams/wes/G41760/U178/current/U178.bam',
 'gs://ccle_bams/rna/G32831/JHH-7/v2/JHH-7.bam',
 'gs://ccle_bams/rna/G16640/COV504/current/COV504.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2171/v1/NCI-H2171.bam',
 'gs://ccle_bams/rna/G16640/SNU-308/current/SNU-308.bam',
 'gs://ccle_bams/wes/G106825/TTC-709/current/TTC-709.bam',
 'gs://ccle_bams/rna/G16640/TE-4/current/TE-4.bam',
 'gs://ccle_bams/rna/G16372/SNU-475/v1/SNU-475.bam',
 'gs://ccle_bams/rna/C1926/KCL-22/current/KCL-22.bam',
 'gs://ccle_bams/rna/G16640/KMBC-2/current/KMBC-2.bam',
 'gs://ccle_bams/rna/G16640/KYSE-150/current/KYSE-150.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2023/current/NCI-H2023.bam',
 'gs://ccle_bams/rna/G16640/Hs_698.T/current/Hs_698.T.bam',
 'gs://ccle_bams/rna/G16367/SK-MES-1/v2/SK-MES-1.bam',
 'gs://ccle_bams/rna/G16640/OVKATE/current/OVKATE.bam',
 'gs://ccle_bams/rna/G29750/TALL-1/v1/TALL-1.bam',
 'gs://ccle_bams/rna/G16640/NUGC-4/current/NUGC-4.bam',
 'gs://ccle_bams/rna/G29750/SK-N-BE_2_/v2/SK-N-BE_2_.bam',
 'gs://ccle_bams/rna/G16640/MC116/current/MC116.bam',
 'gs://ccle_bams/rna/G29750/NU-DUL-1/v2/NU-DUL-1.bam',
 'gs://ccle_bams/rna/G16640/SCC-15/current/SCC-15.bam',
 'gs://ccle_bams/rna/G29750/769-P/v1/769-P.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1105/current/NCI-H1105.bam',
 'gs://ccle_bams/rna/G16640/LN-18/current/LN-18.bam',
 'gs://ccle_bams/wes/G130264/CME-1_1/current/CME-1_1.bam',
 'gs://ccle_bams/rna/G29750/RKN/v3/RKN.bam',
 'gs://ccle_bams/rna/G16640/CAL-120/current/CAL-120.bam',
 'gs://ccle_bams/wes/G141708/LAN2/v2/LAN2.bam',
 'gs://ccle_bams/rna/G16640/HCC-1438/current/HCC-1438.bam',
 'gs://ccle_bams/rna/G29750/HUP-T4/v1/HUP-T4.bam',
 'gs://ccle_bams/rna/G16640/NCI-H661/current/NCI-H661.bam',
 'gs://ccle_bams/wes/G106825/CW9019_1/current/CW9019_1.bam',
 'gs://ccle_bams/rna/G16469/Hs_746T/v1/Hs_746T.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1648/current/NCI-H1648.bam',
 'gs://ccle_bams/rna/G16640/K029AX/current/K029AX.bam',
 'gs://ccle_bams/rna/G29750/SW480/v4/SW480.bam',
 'gs://ccle_bams/rna/G16640/KP-N-SI9s/current/KP-N-SI9s.bam',
 'gs://ccle_bams/rna/G29750/SW48/v1/SW48.bam',
 'gs://ccle_bams/rna/G32831/HEL_92.1.7/v2/HEL_92.1.7.bam',
 'gs://ccle_bams/wes/G27268.SK-HEP-1.1.bam',
 'gs://ccle_bams/rna/G16640/MG-63/current/MG-63.bam',
 'gs://ccle_bams/rna/G16640/FTC-133/current/FTC-133.bam',
 'gs://ccle_bams/rna/G48942/HEL/current/HEL.bam',
 'gs://ccle_bams/rna/G16640/COLO-680N/current/COLO-680N.bam',
 'gs://ccle_bams/rna/G16447/MKN7/v2/MKN7.bam',
 'gs://ccle_bams/rna/G16640/NB-1/current/NB-1.bam',
 'gs://ccle_bams/rna/C1926/LS1034/current/LS1034.bam',
 'gs://ccle_bams/rna/G32831/SW48/v2/SW48.bam',
 'gs://ccle_bams/rna/G32831/PANC-1/v2/PANC-1.bam',
 'gs://ccle_bams/rna/G16443/OVSAHO/v1/OVSAHO.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1915/current/NCI-H1915.bam',
 'gs://ccle_bams/rna/G29750/BICR_22/v2/BICR_22.bam',
 'gs://ccle_bams/wes/G143832/DL_/current/DL_.bam',
 'gs://ccle_bams/rna/G16640/SH-10-TC/current/SH-10-TC.bam',
 'gs://ccle_bams/rna/G32831/BT-483/v1/BT-483.bam',
 'gs://ccle_bams/wes/G91718/FEPD_1/current/FEPD_1.bam',
 'gs://ccle_bams/rna/G29750/SK-MEL-31/v2/SK-MEL-31.bam',
 'gs://ccle_bams/rna/G16446/A2780/v1/A2780.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1092/current/NCI-H1092.bam',
 'gs://ccle_bams/wes/G134624/MCC26_1/current/MCC26_1.bam',
 'gs://ccle_bams/rna/G29750/CAL-33/v2/CAL-33.bam',
 'gs://ccle_bams/rna/C1926/LK-2/current/LK-2.bam',
 'gs://ccle_bams/rna/C817/TC-71/current/TC-71.bam',
 'gs://ccle_bams/rna/G29750/KG-1-C/v2/KG-1-C.bam',
 'gs://ccle_bams/rna/G48942/KMRC-1/current/KMRC-1.bam',
 'gs://ccle_bams/rna/G29750/JHUEM-2/v2/JHUEM-2.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1563/v3/NCI-H1563.bam',
 'gs://ccle_bams/rna/G16640/SK-ES-1/current/SK-ES-1.bam',
 'gs://ccle_bams/rna/G16640/GDM-1/current/GDM-1.bam',
 'gs://ccle_bams/rna/C1926/SNU-761/current/SNU-761.bam',
 'gs://ccle_bams/rna/G32831/NCI-H520/v1/NCI-H520.bam',
 'gs://ccle_bams/rna/G48942/HCC-1438/current/HCC-1438.bam',
 'gs://ccle_bams/rna/G32831/TE-1/v1/TE-1.bam',
 'gs://ccle_bams/wes/G41787.HEK_TE.bam',
 'gs://ccle_bams/rna/G32831/HT115/v2/HT115.bam',
 'gs://ccle_bams/rna/G29750/HCC1937/v2/HCC1937.bam',
 'gs://ccle_bams/rna/C1926/EPLC-272H/current/EPLC-272H.bam',
 'gs://ccle_bams/wes/G106825/CHLA-9_1/current/CHLA-9_1.bam',
 'gs://ccle_bams/rna/G16640/UACC-257/current/UACC-257.bam',
 'gs://ccle_bams/rna/G29750/HEC-251/v3/HEC-251.bam',
 'gs://ccle_bams/wes/G41813/A427/current/A427.bam',
 'gs://ccle_bams/rna/G16640/LS_180/current/LS_180.bam',
 'gs://ccle_bams/rna/G16640/NCI-H524/current/NCI-H524.bam',
 'gs://ccle_bams/wes/G144159/EGI1/v1/EGI1.bam',
 'gs://ccle_bams/rna/C818/CR_G-401/current/CR_G-401.bam',
 'gs://ccle_bams/wes/G141708/PACADD135/v2/PACADD135.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2029/current/NCI-H2029.bam',
 'gs://ccle_bams/rna/C1926/RERF-LC-Ad1/current/RERF-LC-Ad1.bam',
 'gs://ccle_bams/rna/G16640/VMRC-RCW/current/VMRC-RCW.bam',
 'gs://ccle_bams/rna/G29750/L-363/v2/L-363.bam',
 'gs://ccle_bams/rna/G32831/NCI-H1693/v1/NCI-H1693.bam',
 'gs://ccle_bams/rna/G16640/SW_1088/current/SW_1088.bam',
 'gs://ccle_bams/wes/G100662/BIN-67_1/current/BIN-67_1.bam',
 'gs://ccle_bams/rna/G16640/MONO-MAC-1/current/MONO-MAC-1.bam',
 'gs://ccle_bams/rna/G29750/SNU-398/v2/SNU-398.bam',
 'gs://ccle_bams/rna/G29750/SNU-C4/v2/SNU-C4.bam',
 'gs://ccle_bams/rna/G16444/G-401/v1/G-401.bam',
 'gs://ccle_bams/rna/C1926/COR-L95/current/COR-L95.bam',
 'gs://ccle_bams/rna/G16402/NCI-H522/v1/NCI-H522.bam',
 'gs://ccle_bams/rna/G16640/KE-37/current/KE-37.bam',
 'gs://ccle_bams/rna/G29750/HDQ-P1/v1/HDQ-P1.bam',
 'gs://ccle_bams/rna/G29750/GA-10/v2/GA-10.bam',
 'gs://ccle_bams/rna/G16640/SUP-B15/current/SUP-B15.bam',
 'gs://ccle_bams/rna/G29750/COV362/v1/COV362.bam',
 'gs://ccle_bams/rna/G29750/ECC10/v1/ECC10.bam',
 'gs://ccle_bams/rna/G29750/OVK18/v2/OVK18.bam',
 'gs://ccle_bams/rna/C1926/K029AX/current/K029AX.bam',
 'gs://ccle_bams/rna/G16640/COR-L311/current/COR-L311.bam',
 'gs://ccle_bams/rna/G16640/Hey-A8/current/Hey-A8.bam',
 'gs://ccle_bams/rna/G16640/SBC-5/current/SBC-5.bam',
 'gs://ccle_bams/rna/C1926/SW1463/current/SW1463.bam',
 'gs://ccle_bams/rna/G16640/SU-DHL-10/current/SU-DHL-10.bam',
 'gs://ccle_bams/rna/G29750/LUDLU-1/v1/LUDLU-1.bam',
 'gs://ccle_bams/rna/G16353/COR-L88/v1/COR-L88.bam',
 'gs://ccle_bams/rna/C836/T1-73/v1/T1-73.bam',
 'gs://ccle_bams/rna/G16439/ONS-76/v1/ONS-76.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2126/v2/NCI-H2126.bam',
 'gs://ccle_bams/rna/G29750/KYSE-70/v1/KYSE-70.bam',
 'gs://ccle_bams/rna/G29750/SNU-738/v2/SNU-738.bam',
 'gs://ccle_bams/rna/C1926/KMS-27/current/KMS-27.bam',
 'gs://ccle_bams/wes/G134624/A431_1/v4/A431_1.bam',
 'gs://ccle_bams/rna/G29750/Hs_729/v3/Hs_729.bam',
 'gs://ccle_bams/rna/G16640/WM-115/current/WM-115.bam',
 'gs://ccle_bams/rna/C1926/NCI-H292/current/NCI-H292.bam',
 'gs://ccle_bams/rna/G29750/OCI-M1/v1/OCI-M1.bam',
 'gs://ccle_bams/rna/G29750/MHH-CALL-4/v2/MHH-CALL-4.bam',
 'gs://ccle_bams/rna/G16640/SU-DHL-5/current/SU-DHL-5.bam',
 'gs://ccle_bams/wes/G141708/MS751/v2/MS751.bam',
 'gs://ccle_bams/rna/C836/K-562/v2/K-562.bam',
 'gs://ccle_bams/rna/C1926/TGBC11TKB/current/TGBC11TKB.bam',
 'gs://ccle_bams/rna/G29750/SW1417/v1/SW1417.bam',
 'gs://ccle_bams/rna/G16640/SIMA/current/SIMA.bam',
 'gs://ccle_bams/rna/G16640/CAMA-1/current/CAMA-1.bam',
 'gs://ccle_bams/rna/G16360/TE-11/v1/TE-11.bam',
 'gs://ccle_bams/rna/G16380/COR-L279/v1/COR-L279.bam',
 'gs://ccle_bams/rna/G29750/NCI-H146/v1/NCI-H146.bam',
 'gs://ccle_bams/rna/G16640/NU-DHL-1/current/NU-DHL-1.bam',
 'gs://ccle_bams/rna/G16640/HuCCT1/current/HuCCT1.bam',
 'gs://ccle_bams/rna/G29750/TE-4/v1/TE-4.bam',
 'gs://ccle_bams/rna/G29750/Hep_G2/v1/Hep_G2.bam',
 'gs://ccle_bams/rna/G29750/WSU-DLCL2/v3/WSU-DLCL2.bam',
 'gs://ccle_bams/rna/G29750/NCI-H2172/v4/NCI-H2172.bam',
 'gs://ccle_bams/rna/G16640/BHT-101/current/BHT-101.bam',
 'gs://ccle_bams/rna/C1926/Hs_706.T/current/Hs_706.T.bam',
 'gs://ccle_bams/rna/G32831/CAL-54/v1/CAL-54.bam',
 'gs://ccle_bams/rna/G16640/SNU-46/current/SNU-46.bam',
 'gs://ccle_bams/rna/G16640/Capan-2/current/Capan-2.bam',
 'gs://ccle_bams/wes/G141708/UMUC10/current/UMUC10.bam',
 'gs://ccle_bams/rna/G16640/COV504/current/COV504.bam',
 'gs://ccle_bams/wes/G26187/MONO-MAC-1/v3/MONO-MAC-1.bam',
 'gs://ccle_bams/rna/G29750/M-07e/v3/M-07e.bam',
 'gs://ccle_bams/rna/G16410/ESS-1/v1/ESS-1.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1693/v3/NCI-H1693.bam',
 'gs://ccle_bams/rna/G29750/DMS_114/v1/DMS_114.bam',
 'gs://ccle_bams/rna/G16640/KPL-1/current/KPL-1.bam',
 'gs://ccle_bams/rna/G29750/SNU-489/v2/SNU-489.bam',
 'gs://ccle_bams/rna/G29750/A-498/v2/A-498.bam',
 'gs://ccle_bams/rna/G16640/Set-2/current/Set-2.bam',
 'gs://ccle_bams/rna/G32831/MDA-MB-157/v3/MDA-MB-157.bam',
 'gs://ccle_bams/rna/G16405/NCI-H650/v1/NCI-H650.bam',
 'gs://ccle_bams/rna/G29750/LS1034/v3/LS1034.bam',
 'gs://ccle_bams/rna/G16379/RKO/v2/RKO.bam',
 'gs://ccle_bams/rna/G16640/SNU-840/current/SNU-840.bam',
 'gs://ccle_bams/rna/C1926/HOP-92/current/HOP-92.bam',
 'gs://ccle_bams/rna/C1926/NCI-H1436/current/NCI-H1436.bam',
 'gs://ccle_bams/rna/G32831/CAS-1/v1/CAS-1.bam',
 'gs://ccle_bams/rna/G29750/CAL-51/v1/CAL-51.bam',
 'gs://ccle_bams/rna/G29750/COV318/v2/COV318.bam',
 'gs://ccle_bams/rna/Getz_IBM_CellLines_RNASeqData_21samples_09222017/C1926/NA/HCC-366/v1/HCC-366.bam',
 'gs://ccle_bams/rna/C1926/TC-71/current/TC-71.bam',
 'gs://ccle_bams/rna/G16640/NCI-H28/current/NCI-H28.bam',
 'gs://ccle_bams/wes/G41804.NCI-H2004_RT_2.bam',
 'gs://ccle_bams/rna/G16640/SCC-15/current/SCC-15.bam',
 'gs://ccle_bams/wes/G27228.A101D.1.bam',
 'gs://ccle_bams/rna/G29750/SH-10-TC/v3/SH-10-TC.bam',
 'gs://ccle_bams/rna/G16640/LN-18/current/LN-18.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1299/v1/NCI-H1299.bam',
 'gs://ccle_bams/rna/G16640/CAL-120/current/CAL-120.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2342/current/NCI-H2342.bam',
 'gs://ccle_bams/wes/G134624/SUM149PT_1/current/SUM149PT_1.bam',
 'gs://ccle_bams/wes/G131709/My-La_1/current/My-La_1.bam',
 'gs://ccle_bams/rna/G29750/Panc_03.27/v2/Panc_03.27.bam',
 'gs://ccle_bams/rna/G16640/SW948/current/SW948.bam',
 'gs://ccle_bams/wes/G144159/PACADD159/current/PACADD159.bam',
 'gs://ccle_bams/rna/G16640/KNS-81/current/KNS-81.bam',
 'gs://ccle_bams/rna/G29750/KMS-12-BM/v1/KMS-12-BM.bam',
 'gs://ccle_bams/rna/G29750/SW480/v4/SW480.bam',
 'gs://ccle_bams/rna/G29750/SNU-1196/v2/SNU-1196.bam',
 'gs://ccle_bams/wes/G134624/C75_1/v4/C75_1.bam',
 'gs://ccle_bams/rna/G29750/HCC-1359/v2/HCC-1359.bam',
 'gs://ccle_bams/rna/C1926/COLO-320/current/COLO-320.bam',
 'gs://ccle_bams/rna/G29750/HD-MY-Z/v1/HD-MY-Z.bam',
 'gs://ccle_bams/wes/G141708/JOPACA1/current/JOPACA1.bam',
 'gs://ccle_bams/rna/G16640/Hs_737.T/current/Hs_737.T.bam',
 'gs://ccle_bams/rna/G48942/HEL/current/HEL.bam',
 'gs://ccle_bams/rna/G16447/MKN7/v2/MKN7.bam',
 'gs://ccle_bams/rna/G16640/PCM6/current/PCM6.bam',
 'gs://ccle_bams/rna/G16640/WM983B/current/WM983B.bam',
 'gs://ccle_bams/rna/G16640/HCC38/current/HCC38.bam',
 'gs://ccle_bams/rna/G16640/SK-OV-3/current/SK-OV-3.bam',
 'gs://ccle_bams/rna/G29750/KMS-21BM/v2/KMS-21BM.bam',
 'gs://ccle_bams/rna/G29750/KNS-62/v1/KNS-62.bam',
 'gs://ccle_bams/wes/G41771.LN-319.bam',
 'gs://ccle_bams/rna/G29750/SNU-1214/v4/SNU-1214.bam',
 'gs://ccle_bams/rna/G32831/H4/v2/H4.bam',
 'gs://ccle_bams/rna/G16435/22Rv1/v1/22Rv1.bam',
 'gs://ccle_bams/rna/G48942/U251-MG/current/U251-MG.bam',
 'gs://ccle_bams/rna/C1926/HuG1-N/current/HuG1-N.bam',
 'gs://ccle_bams/rna/G32831/T-47D/v1/T-47D.bam',
 'gs://ccle_bams/rna/G16344/786-O/v2/786-O.bam',
 'gs://ccle_bams/wes/G41777/SLR23/current/SLR23.bam',
 'gs://ccle_bams/rna/G29750/SNU-1197/v2/SNU-1197.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2347/current/NCI-H2347.bam',
 'gs://ccle_bams/rna/C1926/Hs_821.T/current/Hs_821.T.bam',
 'gs://ccle_bams/rna/G32831/EOL-1/v1/EOL-1.bam',
 'gs://ccle_bams/rna/G48942/RCC10RGB/current/RCC10RGB.bam',
 'gs://ccle_bams/rna/G16456/SH-10-TC/v2/SH-10-TC.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1105/current/NCI-H1105.bam',
 'gs://ccle_bams/rna/G29750/Capan-2/v1/Capan-2.bam',
 'gs://ccle_bams/rna/G29750/HCC-15/v1/HCC-15.bam',
 'gs://ccle_bams/rna/G16640/THP-1/current/THP-1.bam',
 'gs://ccle_bams/rna/G29750/JK-1/v4/JK-1.bam',
 'gs://ccle_bams/rna/G16640/SNU-761/current/SNU-761.bam',
 'gs://ccle_bams/rna/C1926/CL-11/current/CL-11.bam',
 'gs://ccle_bams/rna/G32831/MDA-MB-231/v2/MDA-MB-231.bam',
 'gs://ccle_bams/rna/G29750/VMRC-RCZ/v2/VMRC-RCZ.bam',
 'gs://ccle_bams/rna/G29750/HCC1937/v2/HCC1937.bam',
 'gs://ccle_bams/rna/G29750/ACHN/v1/ACHN.bam',
 'gs://ccle_bams/rna/G16640/JM1/current/JM1.bam',
 'gs://ccle_bams/rna/G48942/SNU-C1/current/SNU-C1.bam',
 'gs://ccle_bams/rna/G16640/TT2609-C02/current/TT2609-C02.bam',
 'gs://ccle_bams/rna/G32831/OS-RC-2/v1/OS-RC-2.bam',
 'gs://ccle_bams/rna/G29750/BT-474/v3/BT-474.bam',
 'gs://ccle_bams/rna/C1926/SW837/current/SW837.bam',
 'gs://ccle_bams/rna/G29750/ALL-SIL/v3/ALL-SIL.bam',
 'gs://ccle_bams/rna/G29750/Hs_934.T/v2/Hs_934.T.bam',
 'gs://ccle_bams/rna/C1926/SNU-761/current/SNU-761.bam',
 'gs://ccle_bams/rna/G16640/NCI-H2023/current/NCI-H2023.bam',
 'gs://ccle_bams/rna/G16640/OCUM-1/current/OCUM-1.bam',
 'gs://ccle_bams/rna/G16428/NCI-H1092/v1/NCI-H1092.bam',
 'gs://ccle_bams/rna/G16462/KMS-11/v1/KMS-11.bam',
 'gs://ccle_bams/wes/G28037/KYO-1/v2/KYO-1.bam',
 'gs://ccle_bams/rna/G16455/SCLC-21H/v1/SCLC-21H.bam',
 'gs://ccle_bams/wes/G134624/SEKI_1/current/SEKI_1.bam',
 'gs://ccle_bams/rna/G16640/SNU-308/current/SNU-308.bam',
 'gs://ccle_bams/wes/G131709/C8166/current/C8166.bam',
 'gs://ccle_bams/rna/G16640/RPMI_8226/current/RPMI_8226.bam',
 'gs://ccle_bams/rna/G16640/ZR-75-1/current/ZR-75-1.bam',
 'gs://ccle_bams/rna/G16640/HCC2935/current/HCC2935.bam',
 'gs://ccle_bams/rna/G16640/G-401/current/G-401.bam',
 'gs://ccle_bams/rna/G29750/TOV-21G/v5/TOV-21G.bam',
 'gs://ccle_bams/rna/G32831/ACHN/v2/ACHN.bam',
 'gs://ccle_bams/wes/G28831.J82.3.bam',
 'gs://ccle_bams/wes/G143832/HA1E/v1/HA1E.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1930/v2/NCI-H1930.bam',
 'gs://ccle_bams/rna/G29750/MDA-MB-453/v1/MDA-MB-453.bam',
 'gs://ccle_bams/rna/G29750/RERF-LC-Ad1/v2/RERF-LC-Ad1.bam',
 'gs://ccle_bams/rna/G16640/HPB-ALL/current/HPB-ALL.bam',
 'gs://ccle_bams/rna/G29750/RMUG-S/v3/RMUG-S.bam',
 'gs://ccle_bams/rna/G16433/MM1-S/v1/MM1-S.bam',
 'gs://ccle_bams/rna/C1926/NCI-H2405/current/NCI-H2405.bam',
 'gs://ccle_bams/wes/G144963/SW954/v2/SW954.bam',
 'gs://ccle_bams/wes/G35076/NCI-H1373/current/NCI-H1373.bam',
 'gs://ccle_bams/rna/G29750/KE-39/v1/KE-39.bam',
 'gs://ccle_bams/rna/G16640/JHUEM-2/current/JHUEM-2.bam',
 'gs://ccle_bams/rna/G29750/SU-DHL-10/v4/SU-DHL-10.bam',
 'gs://ccle_bams/wes/G25205.NCI-H2286.1.bam',
 'gs://ccle_bams/rna/G16640/SNU-398/current/SNU-398.bam',
 'gs://ccle_bams/wes/G131767/SMZ-1_2/current/SMZ-1_2.bam',
 'gs://ccle_bams/rna/G16477/RERF-GC-1B/v1/RERF-GC-1B.bam',
 'gs://ccle_bams/rna/C1926/RERF-LC-AI/current/RERF-LC-AI.bam',
 'gs://ccle_bams/rna/G29750/697/v2/697.bam',
 'gs://ccle_bams/rna/G29750/SU-DHL-6/v2/SU-DHL-6.bam',
 'gs://ccle_bams/rna/G16640/NCI-H929/current/NCI-H929.bam',
 'gs://ccle_bams/rna/G29750/PC-3/v1/PC-3.bam',
 'gs://ccle_bams/wes/G106825/CHLA266_1/current/CHLA266_1.bam',
 'gs://ccle_bams/rna/G16640/NCI-H1650/current/NCI-H1650.bam',
 'gs://ccle_bams/rna/G16640/KYM-1/current/KYM-1.bam',
 'gs://ccle_bams/rna/G16640/OVISE/current/OVISE.bam',
 'gs://ccle_bams/rna/G16640/SNU-1076/current/SNU-1076.bam',
 'gs://ccle_bams/rna/G16640/NUGC-4/current/NUGC-4.bam',
 'gs://ccle_bams/rna/G29750/BT-20/v1/BT-20.bam',
 'gs://ccle_bams/rna/G16640/5637/current/5637.bam',
 'gs://ccle_bams/rna/G29750/DMS_153/v1/DMS_153.bam',
 'gs://ccle_bams/rna/G29750/HEL_92.1.7/v2/HEL_92.1.7.bam',
 'gs://ccle_bams/rna/G29750/ZR-75-1/v2/ZR-75-1.bam',
 'gs://ccle_bams/rna/G16640/DU4475/current/DU4475.bam',
 'gs://ccle_bams/rna/G16640/PC-3/current/PC-3.bam',
 'gs://ccle_bams/rna/G29750/253J/v2/253J.bam',
 'gs://ccle_bams/rna/C1926/COR-L47/current/COR-L47.bam',
 'gs://ccle_bams/rna/G29750/HSC-3/v2/HSC-3.bam',
 'gs://ccle_bams/rna/G29750/GAMG/v1/GAMG.bam',
 'gs://ccle_bams/rna/G16640/SNU-668/current/SNU-668.bam',
 'gs://ccle_bams/wes/G35068/SNU-283/current/SNU-283.bam',
 'gs://ccle_bams/rna/G29750/NCI-H1395/v2/NCI-H1395.bam',
 'gs://ccle_bams/wes/Getz_IBM_CellLines_Exomes_11samples_04122018/G141708/NA/SISO/v2/SISO.bam',
 'gs://ccle_bams/rna/G16640/NCI-H524/current/NCI-H524.bam']

In [54]:
import dalmatian as dm

In [137]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')
sam = wm.get_samples()

In [138]:
sam.loc['ACH-000150_2'].RNAseq_bam

'gs://ccle_bams/rna/G28870.HCC2935.3.bam'

In [140]:
likelymisslabinrna = set(sam.RNAseq_bam) &set(likelymisslab)

they might be wes!

In [141]:
likelymisslabinrna = [i for i in likelymisslabinrna if "/rna/" in i]
likelymisslabinrna

[]

In [157]:
name = !gsutil ls gs://ccle_bams/rna/**/*SW_1573*.bam
set(name) - set(likelymisslab)

{'gs://ccle_bams/rna/G30620.SW_1573.1.bam'}

In [158]:
name

['gs://ccle_bams/rna/G29750/SW_1573/v2/SW_1573.bam',
 'gs://ccle_bams/rna/G30620.SW_1573.1.bam',
 'gs://ccle_bams/rna/Getz_IBM_CellLines_RNASeqData_1sample_09262017/C1926/NA/SW_1573/v1/SW_1573.bam']

In [133]:
sam[sam.RNAseq_bam.isin(likelymisslabinrna)].index

Index(['ACH-000150_2', 'ACH-000677_2', 'ACH-000840_2', 'ACH-001147_1'], dtype='object', name='sample_id')

In [118]:
sam[sam.RNAseq_bam.isin(likelymisslabinrna)].RNAseq_bam.values

array(['gs://ccle_bams/rna/C1926/HCC2935/current/HCC2935.bam',
       'gs://ccle_bams/rna/Getz_IBM_CellLines_RNASeqData_1sample_09262017/C1926/NA/SW_1573/v1/SW_1573.bam',
       'gs://ccle_bams/rna/Getz_IBM_CellLines_RNASeqData_21samples_09222017/C1926/NA/HCC-366/v1/HCC-366.bam',
       'gs://ccle_bams/rna/G29750/LU99/v1/LU99.bam'], dtype=object)

In [136]:
bams = ['gs://ccle_bams/rna/G28870.HCC2935.3.bam',
      'gs://ccle_bams/rna/G30620.SW_1573.1.bam',
         'gs://ccle_bams/rna/G28840.HCC-366.3.bam',
        'gs://ccle_bams/rna/G28021.LU99.1.bam'
        ]

bais = ['gs://ccle_bams/rna/G28870.HCC2935.3.bai',
    'gs://ccle_bams/rna/G30620.SW_1573.1.bam.bai',
     'gs://ccle_bams/rna/G28840.HCC-366.3.bai',
    'gs://ccle_bams/rna/G28021.LU99.1.bam.bai'
            ]

sids = ['ACH-000150_2', 'ACH-000677_2', 'ACH-000840_2', 'ACH-001147_1']
for i in range(len(bais)):
    wm.update_sample_attributes({'RNAseq_bam':bams[i],'RNAseq_bai': bais[i]},sids[i])

Successfully updated attributes '['RNAseq_bam', 'RNAseq_bai']' for 1 samples.
Hound executing batch upload of 6 records
Successfully updated attributes '['RNAseq_bam', 'RNAseq_bai']' for 1 samples.
Hound executing batch upload of 6 records
Successfully updated attributes '['RNAseq_bam', 'RNAseq_bai']' for 1 samples.
Hound executing batch upload of 6 records
Successfully updated attributes '['RNAseq_bam', 'RNAseq_bai']' for 1 samples.
Hound executing batch upload of 6 records


In [75]:
len(likelymisslabinrna)

144

we need to find the rna version of likelymisslabinrna that are said to be wes (they are rna files that seem to contain wes data).

In [72]:
inrel = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq').get_sample_sets().loc['20Q1'].samples

In [73]:
inrel

['ACH-001847_1',
 'ACH-001533_1',
 'ACH-002014_1',
 'ACH-001574_1',
 'ACH-001625_1']

In [74]:
set(sam[sam.RNAseq_bam.isin(likelymisslabinrna)].index) & set(inrel)

set()

In [226]:
cnwm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WES_CN_hg38')
sam = cnwm.get_samples()

In [79]:
likelymisslabincn = set(cnwm.WES_bam) &set(likelymisslab)

In [81]:
likelymisslabincn

{'gs://ccle_bams/wes/RP-1561/Exome/NOZ/current/NOZ.bam',
 'gs://ccle_bams/wes/U343/current/U343.bam'}

In [161]:
bams = !gsutil ls gs://ccle_bams/wes/**NOZ*.bam
set(bams) - set(likelymisslab)

{'gs://ccle_bams/wes/RP-1561/Exome/NOZ/v1/NOZ.bam'}

In [ ]:
inRNAreplace =  
{"gs://ccle_bams/wes/G41763/U343/current/U343.bam":
"gs://ccle_bams/wes/U343/current/U343.bam",
"gs://ccle_bams/wes/C1926/SNU-601/current/SNU-601.bam":
"gs://ccle_bams/rna/G27466.SNU-601.2.bam"}

inWESreplace = 
{'gs://ccle_bams/wes/U343/current/U343.bam':
'gs://ccle_bams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/U343/v1/U343.bam',
"gs://ccle_bams/wes/RP-1561/Exome/NOZ/current/NOZ.bam":
"gs://ccle_bams/wes/RP-1561/Exome/NOZ/v1/NOZ.bam",
}

In [227]:
mutwm= dm.WorkspaceManager('broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline')
sam = mutwm.get_samples()

In [83]:
likelymisslabinmut = set(mutwm.WES_bam) &set(likelymisslab)

In [162]:
likelymisslabinmut

{'gs://ccle_bams/wes/RP-1561/Exome/NOZ/current/NOZ.bam',
 'gs://ccle_bams/wes/U343/current/U343.bam'}

they are the same in both

In [72]:
!gsutil ls gs://ccle_bams/rna/**/*U343*

gs://ccle_bams/rna/G41763/U343/current/U343.bai
gs://ccle_bams/rna/G41763/U343/current/U343.bam.md5


In [76]:
likely_wes= []
for val in ["gs://ccle_bams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/U343/v1/U343.bam",
            "gs://ccle_bams/wes/U343/current/U343.bam"]:
    res = os.popen('samtools view -H '+val).read()
    if 'bwa mem' in res:
        print(val)

In [54]:
inrna = [i for i in sizes.keys() if '/rna/' in i]

In [56]:
inrna_name = [i.split('/')[-1] for i in inrna]

In [58]:
set(inRNAbutwesfolder_name) & set(inrna_name)

{'G27206.SNU-201.1.bam',
 'G28543.OCI-LY-19.1.bam',
 'Hs_751.T.bam',
 'KYO-1.bam',
 'MONO-MAC-1.bam',
 'NCI-H1373.bam',
 'OVKATE.bam',
 'SNU-283.bam',
 'SNU-601.bam'}

In [ ]:
gs://ccle_bams/wes/SANGER_NCIH211_LUNG.bam seems to have a bad header

# IN MUTATION BUT NOT IN CN

In [171]:
len(inmutbnotincn)

14

In [142]:
inmutbnotincn={'gs://ccle_bams/wes/C835.HCC1143_BL.4.bam',
 'gs://ccle_bams/wes/CCLF_RCRF1009GL.bam',
 'gs://ccle_bams/wes/LPS067.bam',
 'gs://ccle_bams/wes/MUTZ8.bam',
 'gs://ccle_bams/wes/OCIAML4.bam',
 'gs://ccle_bams/wes/OMM1.bam',
 'gs://ccle_bams/wes/R256.bam',
 'gs://ccle_bams/wes/RPE1SS111.bam',
 'gs://ccle_bams/wes/RPE1SS119.bam',
 'gs://ccle_bams/wes/RPE1SS48.bam',
 'gs://ccle_bams/wes/RPE1SS51.bam',
 'gs://ccle_bams/wes/RPE1SS6.bam',
 'gs://ccle_bams/wes/RPE1SS77.bam',
 'gs://ccle_bams/wes/WM3211.bam'}

In [147]:
set(likelymisslab) & inmutbnotincn

set()

{'gs://ccle_bams/wes/C835.HCC1143_BL.4.bam',
 'gs://ccle_bams/wes/CCLF_RCRF1009GL.bam'}

In [170]:
[keeprem[i] for i in (set(inmutbnotincn) & set(keeprem.keys()))]

['gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/MUTZ8/v1/MUTZ8.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch2_Sept2019/RP-1561/Exome/OCIAML4/v1/OCIAML4.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Sept2019/RP-1561/Exome/LPS067/v1/LPS067.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch2_June2019/RP-1561/Exome/OMM1/v1/OMM1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS51/v1/RPE1SS51.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS6/v1/RPE1SS6.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS77/v1/RPE1SS77.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS119/v1/RPE1SS119.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Sept2019/RP-1561/Exome/WM3211/v1/WM3211.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch3_June2019/RP-1561/Exome/R256/v2/R256.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS111/v1/RPE1SS111.bam',
 'gs://c

In [183]:
addtoCNfromMut = set(inmutbnotincn) - set(keeprem.keys())

In [ ]:
tochangeinCN = {
 'gs://ccle_bams/wes/LPS067.bam':'gs://ccle_bams/wes/DepMap_CellLine_WES_Sept2019/RP-1561/Exome/LPS067/v1/LPS067.bam',
 'gs://ccle_bams/wes/MUTZ8.bam':'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/MUTZ8/v1/MUTZ8.bam',
 'gs://ccle_bams/wes/OCIAML4.bam':'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch2_Sept2019/RP-1561/Exome/OCIAML4/v1/OCIAML4.bam',
 'gs://ccle_bams/wes/OMM1.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch2_June2019/RP-1561/Exome/OMM1/v1/OMM1.bam',
 'gs://ccle_bams/wes/R256.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch3_June2019/RP-1561/Exome/R256/v2/R256.bam',
 'gs://ccle_bams/wes/RPE1SS111.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS111/v1/RPE1SS111.bam',
 'gs://ccle_bams/wes/RPE1SS119.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS119/v1/RPE1SS119.bam',
 'gs://ccle_bams/wes/RPE1SS48.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS48/v1/RPE1SS48.bam',
 'gs://ccle_bams/wes/RPE1SS51.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS51/v1/RPE1SS51.bam',
 'gs://ccle_bams/wes/RPE1SS6.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS6/v1/RPE1SS6.bam',
 'gs://ccle_bams/wes/RPE1SS77.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS77/v1/RPE1SS77.bam',
 'gs://ccle_bams/wes/WM3211.bam': 'gs://ccle_bams/wes/DepMap_CellLine_WES_Sept2019/RP-1561/Exome/WM3211/v1/WM3211.bam'
}

In [176]:
set(inmutbnotincn) & set(keeprem.values())

set()

# IN CN BUT NOT IN MUT

In [146]:
keeprem

{'gs://ccle_bams/rna/94T778.bam': 'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/94T778/v3/94T778.bam',
 'gs://ccle_bams/rna/C396.bam': 'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/C396/v2/C396.bam',
 'gs://ccle_bams/rna/CCLP1.bam': 'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/CCLP1/v1/CCLP1.bam',
 'gs://ccle_bams/rna/CII.bam': 'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/CII/v3/CII.bam',
 'gs://ccle_bams/rna/Becker.bam': 'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Aug2019/RP-1561/RNA/Becker/v1/Becker.bam',
 'gs://ccle_bams/rna/CCLFUPGI0005T.bam': 'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Batch2_July2019/RP-1561/RNA/CCLFUPGI0005T/v2/CCLFUPGI0005T.bam',
 'gs://ccle_bams/rna/G415.bam': 'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/G415/v1/G415.bam',
 'gs://ccle_bams/rna/GB2.bam': 'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_July2019/RP-1561/RNA/GB2/v2/GB2.bam',
 'gs://ccle_bams/rna/A388.bam': '

In [143]:
incnbnotinmut={'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch2_June2019/RP-1561/Exome/OMM1/v1/OMM1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch2_Sept2019/RP-1561/Exome/OCIAML4/v1/OCIAML4.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch3_June2019/RP-1561/Exome/R256/v1/R256.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch3_June2019/RP-1561/Exome/R256/v2/R256.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/GB2/v1/GB2.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HAP1MLH1KO_1/v1/HAP1MLH1KO_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HAP1_1/v1/HAP1_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HSQ89_1/v1/HSQ89_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HSSCH2_1/v1/HSSCH2_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/NALM16/v1/NALM16.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/MUTZ8/v1/MUTZ8.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS111/v1/RPE1SS111.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS119/v1/RPE1SS119.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS48/v1/RPE1SS48.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS51/v1/RPE1SS51.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS6/v1/RPE1SS6.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_June2019/RP-1561/Exome/RPE1SS77/v1/RPE1SS77.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Sept2019/RP-1561/Exome/LPS067/v1/LPS067.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_Sept2019/RP-1561/Exome/WM3211/v1/WM3211.bam',
 'gs://ccle_bams/wes/v2/PFSK1.bam',
 'gs://ccle_bams/wes/v2/WERIRB1.bam'}

In [174]:
set(likelymisslab) & incnbnotinmut

set()

In [187]:
addtoCNfromMut = set(incnbnotinmut) - set(keeprem.values())
addtoCNfromMut = addtoCNfromMut - lostcn
addtoCNfromMut

{'gs://ccle_bams/wes/v2/PFSK1.bam', 'gs://ccle_bams/wes/v2/WERIRB1.bam'}

## lost CN datat for  

In [193]:
dup

[('gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/94T778/v3/94T778.bam',
  'gs://ccle_bams/rna/94T778.bam'),
 ('gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/C396/v2/C396.bam',
  'gs://ccle_bams/rna/C396.bam'),
 ('gs://ccle_bams/rna/CCLP1.bam',
  'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/CCLP1/v1/CCLP1.bam'),
 ('gs://ccle_bams/rna/CII.bam',
  'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/CII/v3/CII.bam'),
 ('gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Aug2019/RP-1561/RNA/Becker/v1/Becker.bam',
  'gs://ccle_bams/rna/Becker.bam'),
 ('gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Batch2_July2019/RP-1561/RNA/CCLFUPGI0005T/v2/CCLFUPGI0005T.bam',
  'gs://ccle_bams/rna/CCLFUPGI0005T.bam'),
 ('gs://ccle_bams/rna/G415.bam',
  'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/G415/v1/G415.bam'),
 ('gs://ccle_bams/rna/GB2.bam',
  'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_July2019/RP-1561/RNA/GB2/v2/GB2.bam'),
 

In [195]:
[i[1] for i in dup if i[0] in lostcn] + [i[0] for i in dup if i[1] in lostcn]

[]

In [185]:
lostcn={'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch3_June2019/RP-1561/Exome/R256/v1/R256.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/GB2/v1/GB2.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HAP1MLH1KO_1/v1/HAP1MLH1KO_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HAP1_1/v1/HAP1_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HSQ89_1/v1/HSQ89_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HSSCH2_1/v1/HSSCH2_1.bam',
 'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/NALM16/v1/NALM16.bam',
 'gs://ccle_bams/wes/SANGER_J82_URINARY_TRACT.bam',
 'gs://ccle_bams/wes/SANGER_PA1_OVARY.bam',
 'gs://ccle_bams/wes/SANGER_TE11_OESOPHAGUS.bam',
 'gs://ccle_bams/wes/chordomafoundationsequencedata/BAM_Files/ExomeSeq/ExomeSeq/U-CH1_ExomeSeq_GS001437.cleaned.rg.sorted.markedDupes.realigned.recal.bam',
 'gs://ccle_bams/wes/chordomafoundationsequencedata/BAM_Files/ExomeSeq/ExomeSeq/U-CH2_ExomeSeq_GS001438.cleaned.rg.sorted.markedDupes.realigned.recal.bam'}

## lost mut data for 

In [ ]:
lostmut={'gs://ccle_bams/wes/SANGER_J82_URINARY_TRACT.bam',
 'gs://ccle_bams/wes/SANGER_PA1_OVARY.bam',
 'gs://ccle_bams/wes/SANGER_TE11_OESOPHAGUS.bam',
 'gs://ccle_bams/wes/chordomafoundationsequencedata/BAM_Files/ExomeSeq/ExomeSeq/U-CH1_ExomeSeq_GS001437.cleaned.rg.sorted.markedDupes.realigned.recal.bam',
 'gs://ccle_bams/wes/chordomafoundationsequencedata/BAM_Files/ExomeSeq/ExomeSeq/U-CH2_ExomeSeq_GS001438.cleaned.rg.sorted.markedDupes.realigned.recal.bam'}

## lost RNA data for

In [190]:
lostrna={'gs://ccle_bams/rna/RNA/LPS141/v2/LPS141.bam',
 'gs://ccle_bams/rna/VAESBJ/v2/VAESBJ.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/HCSC1/v1/HCSC1.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/HT144SKINFV1/v1/HT144SKINFV1.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/HT144SKINFV2/v1/HT144SKINFV2.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/HT144SKINFV3/v1/HT144SKINFV3.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/KML1/v1/KML1.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/RVH421SKINFV1/v1/RVH421SKINFV1.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/SKGII/v1/SKGII.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/SNU1544/v1/SNU1544.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/SUM44PE/v1/SUM44PE.bam',
 'gs://fc-8a38e8a8-44ef-44c6-9bb8-4201797c5d7f/Getz_IBM_DMX_CellLines_10samples_Truseq_100M_12082018/RP-1420/RNA/UHO1/v1/UHO1.bam'}

In [196]:
[i[1] for i in dup if i[0] in lostrna] + [i[0] for i in dup if i[1] in lostrna]

[]

In [209]:
adup = [i[0] for i in dup] + [i[1] for i in dup]

In [222]:
!gsutil -m ls -al gs://ccle_bams/**HCSC1*.bam

8862106174  2019-11-13T21:32:09Z  gs://ccle_bams/rna/HCSC1.bam#1573680729325076  metageneration=1
8862106174  2019-10-29T15:44:16Z  gs://ccle_bams/rna/HCSC1/v2/HCSC1.bam#1572363856921432  metageneration=1
22051020828  2019-10-22T21:25:37Z  gs://ccle_bams/wes/Getz_IBM_DMX_CellLines_12samples_deepcov_01302018/RP-1561/Exome/HCSC1/v1/HCSC1.bam#1571779537415657  metageneration=1
TOTAL: 3 objects, 39775233176 bytes (37.04 GiB)


In [213]:
adup

['gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/94T778/v3/94T778.bam',
 'gs://ccle_bams/rna/CCLE_DepMap_RNAseq_July_Dec_2018/RP-1561/RNA/C396/v2/C396.bam',
 'gs://ccle_bams/rna/CCLP1.bam',
 'gs://ccle_bams/rna/CII.bam',
 'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Aug2019/RP-1561/RNA/Becker/v1/Becker.bam',
 'gs://ccle_bams/rna/DepMap_CellLine_RNAseq_Batch2_July2019/RP-1561/RNA/CCLFUPGI0005T/v2/CCLFUPGI0005T.bam',
 'gs://ccle_bams/rna/G415.bam',
 'gs://ccle_bams/rna/GB2.bam',
 'gs://ccle_bams/rna/Getz_IBM_CellLine_RNASeq_March2019/RP-1561/RNA/A388/v3/A388.bam',
 'gs://ccle_bams/rna/Getz_IBM_CellLine_RNAseq_April2019/RP-1561/RNA/BPH1/v1/BPH1.bam',
 'gs://ccle_bams/rna/Getz_IBM_CellLine_RNAseq_June2019/RP-1561/RNA/ECC4/v2/ECC4.bam',
 'gs://ccle_bams/rna/H157.bam',
 'gs://ccle_bams/rna/H357.bam',
 'gs://ccle_bams/rna/H376.bam',
 'gs://ccle_bams/rna/H413.bam',
 'gs://ccle_bams/rna/HAP1MLH1KO.bam',
 'gs://ccle_bams/rna/HCSC1/v2/HCSC1.bam',
 'gs://ccle_bams/rna/HEC116.bam',
 

In [ ]:
inRNAreplace =  
{"gs://ccle_bams/wes/G41763/U343/current/U343.bam":
"gs://ccle_bams/wes/U343/current/U343.bam",
"gs://ccle_bams/wes/C1926/SNU-601/current/SNU-601.bam":
"gs://ccle_bams/rna/G27466.SNU-601.2.bam",
"gs://ccle_bams/rna/VAESBJ/v2/VAESBJ.bam":
"gs://ccle_bams/rna/RNA/VAESBJ/v2/VAESBJ.bam",
"gs://ccle_bams/rna/RNA/LPS141/v2/LPS141.bam":
"gs://ccle_bams/rna/LPS141.bam",
""
}

inWESreplace = 
{'gs://ccle_bams/wes/U343/current/U343.bam':
'gs://ccle_bams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/U343/v1/U343.bam',
"gs://ccle_bams/wes/RP-1561/Exome/NOZ/current/NOZ.bam":
"gs://ccle_bams/wes/RP-1561/Exome/NOZ/v1/NOZ.bam",
"gs://ccle_bams/wes/SANGER_TE11_OESOPHAGUS.bam":
"gs://ccle_bams/wes/Getz_IBM_CellLines_Exomes_19samples_02012018/SANGER_TE11_OESOPHAGUS.bam",
'gs://ccle_bams/wes/SANGER_J82_URINARY_TRACT.bam':
"gs://ccle_bams/wes/Getz_IBM_CellLines_Exomes_19samples_02012018/SANGER_PA1_OVARY.bam",
'gs://ccle_bams/wes/SANGER_PA1_OVARY.bam':
"gs://ccle_bams/wes/Getz_IBM_CellLines_Exomes_19samples_02012018/SANGER_J82_URINARY_TRACT.bam", 
}

# MATCH NORMAL TISSUE


In [172]:
matchnormal = {"ACH-002343_1",
"ACH-002344_1",
"ACH-002346_1",
"ACH-002367_1",
"ACH-002368_1",
"ACH-002369_1",
"ACH-002370_1",
"ACH-002364_1",
"ACH-002365_1",
"ACH-002366_1",
"ACH-002359_1",
"ACH-002360_1",
"ACH-002361_1",
"ACH-002362_1",
"ACH-002347_1",
"ACH-002348_1",
"ACH-002349_1",
"ACH-002350_1",
"ACH-002351_1",
"ACH-002352_1",
"ACH-002353_1",
"ACH-002354_1",
"ACH-002355_1",
"ACH-002356_1",
"ACH-002357_1",
"ACH-002358_1",
"ACH-002363_1",
"ACH-002371_1",
"ACH-002372_1",
"ACH-002373_1",
"ACH-002374_1",
"ACH-002342_1",
"ACH-002383_1",
"ACH-002384_1",
"ACH-002379_1",
"ACH-002380_1",
"ACH-002381_1",
"ACH-002382_1",
"ACH-002375_1",
"ACH-002376_1",
"ACH-002377_1",
"ACH-002378_1"}

In [228]:
cnwm.delete_sample(matchnormal)

Sample set "DM19Q1" (668 samples) successfully updated.
Sample set "DM19Q2" (670 samples) successfully updated.
Sample set "DM19Q2_COMPLETE" (1717 samples) successfully updated.
Sample set "DM19Q2_FULL" (1702 samples) successfully updated.
Sample set "Sanger" (1018 samples) successfully updated.
Sample set "SANGER_NORMALS_MIXED" (2 samples) successfully updated.
Sample(s) {'ACH-002347_1', 'ACH-002356_1', 'ACH-002359_1', 'ACH-002348_1', 'ACH-002349_1', 'ACH-002362_1', 'ACH-002374_1', 'ACH-002382_1', 'ACH-002364_1', 'ACH-002376_1', 'ACH-002383_1', 'ACH-002342_1', 'ACH-002367_1', 'ACH-002380_1', 'ACH-002375_1', 'ACH-002344_1', 'ACH-002361_1', 'ACH-002377_1', 'ACH-002352_1', 'ACH-002373_1', 'ACH-002372_1', 'ACH-002365_1', 'ACH-002343_1', 'ACH-002360_1', 'ACH-002370_1', 'ACH-002379_1', 'ACH-002381_1', 'ACH-002384_1', 'ACH-002357_1', 'ACH-002354_1', 'ACH-002368_1', 'ACH-002378_1', 'ACH-002351_1', 'ACH-002353_1', 'ACH-002346_1', 'ACH-002355_1', 'ACH-002350_1', 'ACH-002358_1', 'ACH-002366_1', 

In [ ]:
mutwm.delete_sample(matchnormal)

## still lost

In [ ]:
# NO Versions at all
'gs://ccle_bams/wes/chordomafoundationsequencedata/BAM_File/ExomeSeq/ExomeSeq/U-CH1_ExomeSeq_GS001437.cleaned.rg.sorted.markedDupes.realigned.recal.bam',
 'gs://ccle_bams/wes/chordomafoundationsequencedata/BAM_Files/ExomeSeq/ExomeSeq/U-CH2_ExomeSeq_GS001438.cleaned.rg.sorted.markedDupes.realigned.recal.bam'

In [ ]:
# missing version
'gs://ccle_bams/wes/DepMap_CellLine_WES_Batch3_June2019/RP-1561/Exome/R256/v1/R256.bam',
'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/GB2/v1/GB2.bam',
'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HAP1MLH1KO_1/v1/HAP1MLH1KO_1.bam',
'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HAP1_1/v1/HAP1_1.bam',
'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HSQ89_1/v1/HSQ89_1.bam',
'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/HSSCH2_1/v1/HSSCH2_1.bam',
'gs://ccle_bams/wes/DepMap_CellLine_WES_July2019/RP-1561/Exome/NALM16/v1/NALM16.bam',